# Effective Sample Size
Meghan Rowan Childs
 
 
This jupyter notebook is used to calculate the effective sample size for each of the calibrations we have run. These include

- orginal calibration
- initial infected and adjusted testing term calibration
- delay of exogenous shock calibration
- non_compliance calibration
- reparameterized test specificity

Effective sample size is calculated because the samples from a Markov Chain are autocorrelated. Since this autocorrelation occurs we need to figure out how many samples we need in order to keep a similar accuracy to if the samples were independent.
## Import Needed Packages

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from chainconsumer import ChainConsumer
import tensorflow_probability as tfp


In [2]:
tf.constant(4)


<tf.Tensor: shape=(), dtype=int32, numpy=4>

## Read in Chains for Each Calibration
Now we want to read in chains for each of the calibrations we want to know the effective sample size for.

In [5]:
og_calibration_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/og calibration rerun march 2024/MCMC_Chain_Fall_Data_All_Params_100_iters_og_rerun_mar_24_[952].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
og_calibration_chain_2 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/og calibration rerun march 2024/MCMC_Chain_Fall_Data_All_Params_100_iters_og_rerun_mar_24_[749].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
og_calibration_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/og calibration rerun march 2024/MCMC_Chain_Fall_Data_All_Params_100_iters_og_rerun_mar_24_[704].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
og_calibration_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/og calibration rerun march 2024/MCMC_Chain_Fall_Data_All_Params_100_iters_og_rerun_mar_24_[378].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
    
initi_infect_adjust_test_terms_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/initial infect rerun march 2024/MCMC_Chain_Fall_Data_All_Params_100_iters_init_infect_rerun_mar_24_[10].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infect'])
initi_infect_adjust_test_terms_chain_2 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/initial infect rerun march 2024/MCMC_Chain_Fall_Data_All_Params_100_iters_init_infect_rerun_mar_24_[495].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infect'])
initi_infect_adjust_test_terms_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/initial infect rerun march 2024/MCMC_Chain_Fall_Data_All_Params_100_iters_init_infect_rerun_mar_24_[523].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infect'])
initi_infect_adjust_test_terms_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/initial infect rerun march 2024/MCMC_Chain_Fall_Data_All_Params_100_iters_init_infect_rerun_mar_24_[653].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infect'])

# delay_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/MCMC Files/MCMC CSV Files/Rerun Delay as Timesteps 4 chains/MCMC_Chain_Fall_Data_All_Params_300_iters_nostochastic_uncertain_delayed_es_new_prior_timstep_fix_4_chains_[152].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infect', 'Delay'])
# delay_chain_2 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/MCMC Files/MCMC CSV Files/Rerun Delay as Timesteps 4 chains/MCMC_Chain_Fall_Data_All_Params_300_iters_nostochastic_uncertain_delayed_es_new_prior_timstep_fix_4_chains_[813].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infect', 'Delay'])
# delay_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/MCMC Files/MCMC CSV Files/Rerun Delay as Timesteps 4 chains/MCMC_Chain_Fall_Data_All_Params_300_iters_nostochastic_uncertain_delayed_es_new_prior_timstep_fix_4_chains_[72].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infect', 'Delay'])
# delay_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/MCMC Files/MCMC CSV Files/Rerun Delay as Timesteps 4 chains/MCMC_Chain_Fall_Data_All_Params_300_iters_nostochastic_uncertain_delayed_es_new_prior_timstep_fix_4_chains_[388].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infect', 'Delay'])

# non_compliance_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/MCMC Files/MCMC CSV Files/1st run non_compliance param/MCMC_Chain_Fall_Data_All_Params_400_iters_nostochastic_non_compliance_4_chains_rerun_tight_prior_rerun3_[609].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infect', 'Delay', 'Non_compliance'])
# non_compliance_chain_2 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/MCMC Files/MCMC CSV Files/1st run non_compliance param/MCMC_Chain_Fall_Data_All_Params_400_iters_nostochastic_non_compliance_4_chains_rerun_tight_prior_rerun3_[181].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infect', 'Delay', 'Non_compliance'])
# non_compliance_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/MCMC Files/MCMC CSV Files/1st run non_compliance param/MCMC_Chain_Fall_Data_All_Params_400_iters_nostochastic_non_compliance_4_chains_rerun_tight_prior_rerun3_[65].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infect', 'Delay', 'Non_compliance'])
# non_compliance_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/MCMC Files/MCMC CSV Files/1st run non_compliance param/MCMC_Chain_Fall_Data_All_Params_400_iters_nostochastic_non_compliance_4_chains_rerun_tight_prior_rerun3_[604].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infect', 'Delay', 'Non_compliance'])
stochastic_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/stochastic rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_stochastic_rerun_mar_24_[97].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
stochastic_chain_2 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/stochastic rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_stochastic_rerun_mar_24_[359].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
stochastic_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/stochastic rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_stochastic_rerun_mar_24_[881].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
stochastic_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/stochastic rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_stochastic_rerun_mar_24_[964].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])

stochastic_sp_reparam_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/stochastic AND sp reparam rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_stochastic_with_sp_reparam_rerun_mar_24_[20].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
stochastic_sp_reparam_chain_2 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/stochastic AND sp reparam rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_stochastic_with_sp_reparam_rerun_mar_24_[28].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
stochastic_sp_reparam_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/stochastic AND sp reparam rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_stochastic_with_sp_reparam_rerun_mar_24_[240].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
stochastic_sp_reparam_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/stochastic AND sp reparam rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_stochastic_with_sp_reparam_rerun_mar_24_[643].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])

sp_reparam_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/SP reparam re run march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_sp_reparam_rerun_mar_24_[390].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
sp_reparam_chain_2 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/SP reparam re run march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_sp_reparam_rerun_mar_24_[423].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
sp_reparam_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/SP reparam re run march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_sp_reparam_rerun_mar_24_[425].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
sp_reparam_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/SP reparam re run march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_sp_reparam_rerun_mar_24_[437].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])

model_exper_no_uncertain_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper no uncertainty rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_no_uncertainty_rerun_mar_24_[277].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
model_exper_no_uncertain_chain_2 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper no uncertainty rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_no_uncertainty_rerun_mar_24_[702].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
model_exper_no_uncertain_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper no uncertainty rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_no_uncertainty_rerun_mar_24_[837].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
model_exper_no_uncertain_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper no uncertainty rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_no_uncertainty_rerun_mar_24_[916].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])

model_exper_uncertain_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper with uncertainity rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_uncertainty_rerun_mar_24_[31].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
model_exper_uncertain_chain_2 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper with uncertainity rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_uncertainty_rerun_mar_24_[70].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
model_exper_uncertain_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper with uncertainity rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_uncertainty_rerun_mar_24_[618].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
model_exper_uncertain_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper with uncertainity rerun march 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_uncertainty_rerun_mar_24_[787].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])

model_exper_rt_fixed_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper fixed rt ar 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_fixed_rt_mar_24_[419].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing',  'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
model_exper_rt_fixed_chain_2 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper fixed rt ar 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_fixed_rt_mar_24_[580].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing',  'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
model_exper_rt_fixed_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper fixed rt ar 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_fixed_rt_mar_24_[608].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
model_exper_rt_fixed_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper fixed rt ar 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_fixed_rt_mar_24_[638].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])

model_exper_rt_fixed_community_params_fixed_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper fixed rt and fixed all es mar 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_fixed_new_infect_fixed_freq_es_RT_fixed_mar_24_[719].csv', names = ['Iter', 'Time to Recovery', 'Percent Advancing',  'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity'])
model_exper_rt_fixed_community_params_fixed_chain_2 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper fixed rt and fixed all es mar 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_fixed_new_infect_fixed_freq_es_RT_fixed_mar_24_[790].csv', names = ['Iter',  'Time to Recovery', 'Percent Advancing',  'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity'])
model_exper_rt_fixed_community_params_fixed_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper fixed rt and fixed all es mar 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_fixed_new_infect_fixed_freq_es_RT_fixed_mar_24_[832].csv', names = ['Iter',  'Time to Recovery', 'Percent Advancing',  'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity'])
model_exper_rt_fixed_community_params_fixed_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper fixed rt and fixed all es mar 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_fixed_new_infect_fixed_freq_es_RT_fixed_mar_24_[864].csv', names = ['Iter',  'Time to Recovery', 'Percent Advancing', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity'])

model_exper_adapted_LL_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper adapted LL Mar 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_adapted_LL_first_run_mar_24_[227].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
model_exper_adapted_LL_chain_2 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper adapted LL Mar 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_adapted_LL_first_run_mar_24_[231].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
model_exper_adapted_LL_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper adapted LL Mar 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_adapted_LL_first_run_mar_24_[471].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
model_exper_adapted_LL_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/model exper adapted LL Mar 24/MCMC_Chain_Fall_Data_All_Params_100_iters_model_exper_adapted_LL_first_run_mar_24_[509].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])


rit_data_adapted_LL_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/rit data adapted LL 0 case change apr 24/MCMC_Chain_Fall_Data_All_Params_100_iters_rit_daya_adapted_LL_new_std_change_apr_24_[54].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
# rit_data_adapted_LL_chain_2 = pd.read_csv(r'', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
rit_data_adapted_LL_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/rit data adapted LL 0 case change apr 24/MCMC_Chain_Fall_Data_All_Params_100_iters_rit_daya_adapted_LL_new_std_change_apr_24_[284].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
rit_data_adapted_LL_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/rit data adapted LL 0 case change apr 24/MCMC_Chain_Fall_Data_All_Params_100_iters_rit_daya_adapted_LL_new_std_change_apr_24_[580].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])

rit_data_adapted_LL_init_infect_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/rit data adapted ll new std and init infect apr 24/MCMC_Chain_Fall_Data_All_Params_100_iters_rit_daya_adapted_LL_new_std_random_sample_initial_params_apr_24_[536].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infected'])
rit_data_adapted_LL_init_infect_chain_2 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/rit data adapted ll new std and init infect apr 24/MCMC_Chain_Fall_Data_All_Params_100_iters_rit_daya_adapted_LL_new_std_random_sample_initial_params_apr_24_[656].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infected'])
rit_data_adapted_LL_init_infect_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/rit data adapted ll new std and init infect apr 24/MCMC_Chain_Fall_Data_All_Params_100_iters_rit_daya_adapted_LL_new_std_random_sample_initial_params_apr_24_[725].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infected'])
rit_data_adapted_LL_init_infect_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/rit data adapted ll new std and init infect apr 24/MCMC_Chain_Fall_Data_All_Params_100_iters_rit_daya_adapted_LL_new_std_random_sample_initial_params_apr_24_[985].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infected'])


rit_data_adapted_LL_init_infect_stochastic_weight_chain_1 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/std version of LL weighting (mutlipled std by 2) sept 24/MCMC_Chain_Fall_Data_All_Params_100_iters_rit_data_stochastic_new_ll_adapted_binom_initial_iso_fixed_new_weighting_[203].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infected', 'Initial Iso'])
rit_data_adapted_LL_init_infect_stochastic_weight_chain_2 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/std version of LL weighting (mutlipled std by 2) sept 24/MCMC_Chain_Fall_Data_All_Params_100_iters_rit_data_stochastic_new_ll_adapted_binom_initial_iso_fixed_new_weighting_[379].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infected', 'Initial Iso'])
rit_data_adapted_LL_init_infect_stochastic_weight_chain_3 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/std version of LL weighting (mutlipled std by 2) sept 24/MCMC_Chain_Fall_Data_All_Params_100_iters_rit_data_stochastic_new_ll_adapted_binom_initial_iso_fixed_new_weighting_[443].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infected', 'Initial Iso'])
rit_data_adapted_LL_init_infect_stochastic_weight_chain_4 = pd.read_csv(r'/Users/meghanchilds/Desktop/Dissertation_Research/COVID-19-Model-Formal-Calibration/MCMC CSV Files/std version of LL weighting (mutlipled std by 2) sept 24/MCMC_Chain_Fall_Data_All_Params_100_iters_rit_data_stochastic_new_ll_adapted_binom_initial_iso_fixed_new_weighting_[725].csv', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock', 'Initial Infected', 'Initial Iso'])


In [7]:
rit_data_adapted_LL_init_infect_stochastic_weight_chain_4

,Iter,Frequency of ES,Time to Recovery,Percent Advancing,Rt,Symptom Case Fatality,Days to Incubation,Test Sensitivity,Test Specificity,New Infections per Shock,Initial Infected,Initial Iso
0,0,8.0,13.969285,18.540193,1.747763,0.001308,8.638653,0.789656,1.243258,10.0,20.0,6.000135
1,1,8.0,13.969285,18.540193,1.747763,0.001308,8.638653,0.789656,1.243258,10.0,20.0,6.000135
2,2,8.0,13.969285,18.540193,1.747763,0.001308,8.638653,0.789656,1.243258,10.0,20.0,6.000135
3,3,8.0,14.016459,18.657945,1.751244,0.001364,8.636378,0.788761,1.243772,11.0,20.0,6.058808
4,4,9.0,13.898688,18.670030,1.723085,0.001400,8.646717,0.790807,1.244652,12.0,21.0,6.072732
...,...,...,...,...,...,...,...,...,...,...,...,...
399995,39995,7.0,13.745396,13.982198,1.027439,0.003503,7.515141,0.765479,1.237050,15.0,4.0,1.881346
399996,39996,8.0,14.116982,14.700256,1.056405,0.002680,8.342877,0.750841,1.241829,16.0,7.0,1.664659
399997,39997,8.0,14.116982,14.700256,1.056405,0.002680,8.342877,0.750841,1.241829,16.0,7.0,1.664659
399998,39998,8.0,14.116982,14.700256,1.056405,0.002680,8.342877,0.750841,1.241829,16.0,7.0,1.664659


In [8]:
del og_calibration_chain_1['Iter']
del og_calibration_chain_2['Iter']
del og_calibration_chain_3['Iter']
del og_calibration_chain_4['Iter']


del initi_infect_adjust_test_terms_chain_1['Iter']
del initi_infect_adjust_test_terms_chain_2['Iter']
del initi_infect_adjust_test_terms_chain_3['Iter']
del initi_infect_adjust_test_terms_chain_4['Iter']

del stochastic_chain_1['Iter']
del stochastic_chain_2['Iter']
del stochastic_chain_3['Iter']
del stochastic_chain_4['Iter']

del stochastic_sp_reparam_chain_1['Iter']
del stochastic_sp_reparam_chain_2['Iter']
del stochastic_sp_reparam_chain_3['Iter']
del stochastic_sp_reparam_chain_4['Iter']

# del delay_chain_1['Iter']
# del delay_chain_2['Iter']
# del delay_chain_3['Iter']
# del delay_chain_4['Iter']

# del non_compliance_chain_1['Iter']
# del non_compliance_chain_2['Iter']
# del non_compliance_chain_3['Iter']
# del non_compliance_chain_4['Iter']

del sp_reparam_chain_1['Iter']
del sp_reparam_chain_2['Iter']
del sp_reparam_chain_3['Iter']
del sp_reparam_chain_4['Iter']

del model_exper_no_uncertain_chain_1['Iter']
del model_exper_no_uncertain_chain_2['Iter']
del model_exper_no_uncertain_chain_3['Iter']
del model_exper_no_uncertain_chain_4['Iter']

del model_exper_uncertain_chain_1['Iter']
del model_exper_uncertain_chain_2['Iter']
del model_exper_uncertain_chain_3['Iter']
del model_exper_uncertain_chain_4['Iter']

del model_exper_rt_fixed_chain_1['Iter'] 
del model_exper_rt_fixed_chain_2['Iter'] 
del model_exper_rt_fixed_chain_3['Iter'] 
del model_exper_rt_fixed_chain_4['Iter'] 

del model_exper_rt_fixed_community_params_fixed_chain_1['Iter'] 
del model_exper_rt_fixed_community_params_fixed_chain_2['Iter'] 
del model_exper_rt_fixed_community_params_fixed_chain_3['Iter'] 
del model_exper_rt_fixed_community_params_fixed_chain_4['Iter'] 

del model_exper_adapted_LL_chain_1['Iter'] 
del model_exper_adapted_LL_chain_2['Iter'] 
del model_exper_adapted_LL_chain_3['Iter'] 
del model_exper_adapted_LL_chain_4['Iter']


del rit_data_adapted_LL_chain_1['Iter'] 
# rit_data_adapted_LL_chain_2 
del rit_data_adapted_LL_chain_3['Iter'] 
del rit_data_adapted_LL_chain_4['Iter'] 

del rit_data_adapted_LL_init_infect_chain_1['Iter'] 
del rit_data_adapted_LL_init_infect_chain_2['Iter']
del rit_data_adapted_LL_init_infect_chain_3['Iter']
del rit_data_adapted_LL_init_infect_chain_4['Iter'] 

del rit_data_adapted_LL_init_infect_stochastic_weight_chain_1['Iter'] 
del rit_data_adapted_LL_init_infect_stochastic_weight_chain_2['Iter']
del rit_data_adapted_LL_init_infect_stochastic_weight_chain_3['Iter']
del rit_data_adapted_LL_init_infect_stochastic_weight_chain_4['Iter'] 


### Save Param Names for Each Calibration

In [9]:
og_names = og_calibration_chain_1.columns
init_infect_adjust_tt_names = initi_infect_adjust_test_terms_chain_1.columns
# delay_names = delay_chain_1.columns
# non_comp_names = non_compliance_chain_1.columns
stochastic_names = stochastic_chain_1.columns
stochastic_sp_repara_names  = stochastic_sp_reparam_chain_1.columns
sp_reparam_names = sp_reparam_chain_1.columns
model_exper_no_uncertain_names = model_exper_no_uncertain_chain_1.columns
model_exper_uncertain_names = model_exper_uncertain_chain_1.columns
model_exper_rt_fixed_chain_names = model_exper_rt_fixed_chain_1.columns
model_exper_rt_fixed_community_params_fixed_names = model_exper_rt_fixed_community_params_fixed_chain_1.columns
model_exper_adapted_LL_names = model_exper_adapted_LL_chain_1.columns
rit_data_adapted_LL_names = rit_data_adapted_LL_chain_1.columns
rit_data_adapted_LL_init_infect_names = rit_data_adapted_LL_init_infect_chain_1.columns
rit_data_adapted_LL_init_infect_stochastic_weight = rit_data_adapted_LL_init_infect_stochastic_weight_chain_1.columns

In [11]:
rit_data_adapted_LL_init_infect_stochastic_weight

Index(['Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt',
       'Symptom Case Fatality', 'Days to Incubation', 'Test Sensitivity',
       'Test Specificity', 'New Infections per Shock', 'Initial Infected',
       'Initial Iso'],
      dtype='object')

## Data Formatting

In [12]:
# setting as numpy arrays for easier use
og_calibration_chain_1 = np.array(og_calibration_chain_1)
og_calibration_chain_2 = np.array(og_calibration_chain_2)
og_calibration_chain_3 = np.array(og_calibration_chain_3)
og_calibration_chain_4 = np.array(og_calibration_chain_4)

initi_infect_adjust_test_terms_chain_1 = np.array(initi_infect_adjust_test_terms_chain_1)
initi_infect_adjust_test_terms_chain_2 = np.array(initi_infect_adjust_test_terms_chain_2)
initi_infect_adjust_test_terms_chain_3 = np.array(initi_infect_adjust_test_terms_chain_3)
initi_infect_adjust_test_terms_chain_4 = np.array(initi_infect_adjust_test_terms_chain_4)

# delay_chain_1 = np.array(delay_chain_1)
# delay_chain_2 = np.array(delay_chain_2)
# delay_chain_3 = np.array(delay_chain_3)
# delay_chain_4 = np.array(delay_chain_4)

# non_compliance_chain_1 = np.array(non_compliance_chain_1)
# non_compliance_chain_2 = np.array(non_compliance_chain_2)
# non_compliance_chain_3 = np.array(non_compliance_chain_3)
# non_compliance_chain_4 = np.array(non_compliance_chain_4)

stochastic_chain_1 = np.array(stochastic_chain_1)
stochastic_chain_2 = np.array(stochastic_chain_2)
stochastic_chain_3 = np.array(stochastic_chain_3)
stochastic_chain_4 = np.array(stochastic_chain_4)

stochastic_sp_reparam_chain_1 = np.array(stochastic_sp_reparam_chain_1)
stochastic_sp_reparam_chain_2 = np.array(stochastic_sp_reparam_chain_2)
stochastic_sp_reparam_chain_3 = np.array(stochastic_sp_reparam_chain_3)
stochastic_sp_reparam_chain_4 = np.array(stochastic_sp_reparam_chain_4)

sp_reparam_chain_1 = np.array(sp_reparam_chain_1)
sp_reparam_chain_2 = np.array(sp_reparam_chain_2)
sp_reparam_chain_3 = np.array(sp_reparam_chain_3)
sp_reparam_chain_4 = np.array(sp_reparam_chain_4)

model_exper_no_uncertain_chain_1 = np.array(model_exper_no_uncertain_chain_1)
model_exper_no_uncertain_chain_2 = np.array(model_exper_no_uncertain_chain_2)
model_exper_no_uncertain_chain_3 = np.array(model_exper_no_uncertain_chain_3)
model_exper_no_uncertain_chain_4 = np.array(model_exper_no_uncertain_chain_4)

model_exper_uncertain_chain_1 = np.array(model_exper_uncertain_chain_1)
model_exper_uncertain_chain_2 = np.array(model_exper_uncertain_chain_2)
model_exper_uncertain_chain_3 = np.array(model_exper_uncertain_chain_3)
model_exper_uncertain_chain_4 = np.array(model_exper_uncertain_chain_4)

model_exper_rt_fixed_chain_1 = np.array(model_exper_rt_fixed_chain_1)
model_exper_rt_fixed_chain_2 = np.array(model_exper_rt_fixed_chain_2)
model_exper_rt_fixed_chain_3 = np.array(model_exper_rt_fixed_chain_3)
model_exper_rt_fixed_chain_4 = np.array(model_exper_rt_fixed_chain_4)

model_exper_rt_fixed_community_params_fixed_chain_1 = np.array(model_exper_rt_fixed_community_params_fixed_chain_1)
model_exper_rt_fixed_community_params_fixed_chain_2 = np.array(model_exper_rt_fixed_community_params_fixed_chain_2)
model_exper_rt_fixed_community_params_fixed_chain_3 = np.array(model_exper_rt_fixed_community_params_fixed_chain_3)
model_exper_rt_fixed_community_params_fixed_chain_4 = np.array(model_exper_rt_fixed_community_params_fixed_chain_4)

model_exper_adapted_LL_chain_1 = np.array(model_exper_adapted_LL_chain_1)
model_exper_adapted_LL_chain_2 = np.array(model_exper_adapted_LL_chain_2)
model_exper_adapted_LL_chain_3 = np.array(model_exper_adapted_LL_chain_3)
model_exper_adapted_LL_chain_4 = np.array(model_exper_adapted_LL_chain_4)


rit_data_adapted_LL_chain_1 = np.array(rit_data_adapted_LL_chain_1)
# rit_data_adapted_LL_chain_2 = pd.read_csv(r'', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
rit_data_adapted_LL_chain_3 = np.array(rit_data_adapted_LL_chain_3)
rit_data_adapted_LL_chain_4 = np.array(rit_data_adapted_LL_chain_4)

rit_data_adapted_LL_init_infect_chain_1 = np.array(rit_data_adapted_LL_init_infect_chain_1)
rit_data_adapted_LL_init_infect_chain_2 = np.array(rit_data_adapted_LL_init_infect_chain_2)
rit_data_adapted_LL_init_infect_chain_3 = np.array(rit_data_adapted_LL_init_infect_chain_3)
rit_data_adapted_LL_init_infect_chain_4 = np.array(rit_data_adapted_LL_init_infect_chain_4)

rit_data_adapted_LL_init_infect_stochastic_weight_chain_1 = np.array(rit_data_adapted_LL_init_infect_stochastic_weight_chain_1)
rit_data_adapted_LL_init_infect_stochastic_weight_chain_2 = np.array(rit_data_adapted_LL_init_infect_stochastic_weight_chain_2)
rit_data_adapted_LL_init_infect_stochastic_weight_chain_3 = np.array(rit_data_adapted_LL_init_infect_stochastic_weight_chain_3)
rit_data_adapted_LL_init_infect_stochastic_weight_chain_4 = np.array(rit_data_adapted_LL_init_infect_stochastic_weight_chain_4)



In [13]:
# setting interger values to be integer
og_calibration_chain_1[:,0] = np.array(og_calibration_chain_1[:,0], dtype = int)
og_calibration_chain_1[:,8] = np.array(og_calibration_chain_1[:,8], dtype = int)

og_calibration_chain_2[:,0] = np.array(og_calibration_chain_2[:,0], dtype = int)
og_calibration_chain_2[:,8] = np.array(og_calibration_chain_2[:,8], dtype = int)

og_calibration_chain_3[:,0] = np.array(og_calibration_chain_3[:,0], dtype = int)
og_calibration_chain_3[:,8] = np.array(og_calibration_chain_3[:,8], dtype = int)

og_calibration_chain_4[:,0] = np.array(og_calibration_chain_4[:,0], dtype = int)
og_calibration_chain_4[:,8] = np.array(og_calibration_chain_4[:,8], dtype = int)

initi_infect_adjust_test_terms_chain_1[:,0] = np.array(initi_infect_adjust_test_terms_chain_1[:,0], dtype = int)
initi_infect_adjust_test_terms_chain_1[:,8] = np.array(initi_infect_adjust_test_terms_chain_1[:,8], dtype = int)
initi_infect_adjust_test_terms_chain_1[:,9] = np.array(initi_infect_adjust_test_terms_chain_1[:,9], dtype = int)

initi_infect_adjust_test_terms_chain_2[:,0] = np.array(initi_infect_adjust_test_terms_chain_2[:,0], dtype = int)
initi_infect_adjust_test_terms_chain_2[:,8] = np.array(initi_infect_adjust_test_terms_chain_2[:,8], dtype = int)
initi_infect_adjust_test_terms_chain_2[:,9] = np.array(initi_infect_adjust_test_terms_chain_2[:,9], dtype = int)

initi_infect_adjust_test_terms_chain_3[:,0] = np.array(initi_infect_adjust_test_terms_chain_3[:,0], dtype = int)
initi_infect_adjust_test_terms_chain_3[:,8] = np.array(initi_infect_adjust_test_terms_chain_3[:,8], dtype = int)
initi_infect_adjust_test_terms_chain_3[:,9] = np.array(initi_infect_adjust_test_terms_chain_3[:,9], dtype = int)

initi_infect_adjust_test_terms_chain_4[:,0] = np.array(initi_infect_adjust_test_terms_chain_4[:,0], dtype = int)
initi_infect_adjust_test_terms_chain_4[:,8] = np.array(initi_infect_adjust_test_terms_chain_4[:,8], dtype = int)
initi_infect_adjust_test_terms_chain_4[:,9] = np.array(initi_infect_adjust_test_terms_chain_4[:,9], dtype = int)

# delay_chain_1[:,0] = np.array(delay_chain_1[:,0], dtype = int)
# delay_chain_1[:,8] = np.array(delay_chain_1[:,8], dtype = int)
# delay_chain_1[:,9] = np.array(delay_chain_1[:,9], dtype = int)
# delay_chain_1[:,10] = np.array(delay_chain_1[:,10], dtype = int)

# delay_chain_2[:,0] = np.array(delay_chain_2[:,0], dtype = int)
# delay_chain_2[:,8] = np.array(delay_chain_2[:,8], dtype = int)
# delay_chain_2[:,9] = np.array(delay_chain_2[:,9], dtype = int)
# delay_chain_2[:,10] = np.array(delay_chain_2[:,10], dtype = int)

# delay_chain_3[:,0] = np.array(delay_chain_3[:,0], dtype = int)
# delay_chain_3[:,8] = np.array(delay_chain_3[:,8], dtype = int)
# delay_chain_3[:,9] = np.array(delay_chain_3[:,9], dtype = int)
# delay_chain_3[:,10] = np.array(delay_chain_3[:,10], dtype = int)

# delay_chain_4[:,0] = np.array(delay_chain_4[:,0], dtype = int)
# delay_chain_4[:,8] = np.array(delay_chain_4[:,8], dtype = int)
# delay_chain_4[:,9] = np.array(delay_chain_4[:,9], dtype = int)
# delay_chain_4[:,10] = np.array(delay_chain_4[:,10], dtype = int)

# non_compliance_chain_1[:,0] = np.array(non_compliance_chain_1[:,0], dtype = int)
# non_compliance_chain_1[:,8] = np.array(non_compliance_chain_1[:,8], dtype = int)
# non_compliance_chain_1[:,9] = np.array(non_compliance_chain_1[:,9], dtype = int)
# non_compliance_chain_1[:,10] = np.array(non_compliance_chain_1[:,10], dtype = int)

# non_compliance_chain_2[:,0] = np.array(non_compliance_chain_2[:,0], dtype = int)
# non_compliance_chain_2[:,8] = np.array(non_compliance_chain_2[:,8], dtype = int)
# non_compliance_chain_2[:,9] = np.array(non_compliance_chain_2[:,9], dtype = int)
# non_compliance_chain_2[:,10] = np.array(non_compliance_chain_2[:,10], dtype = int)

# non_compliance_chain_3[:,0] = np.array(non_compliance_chain_3[:,0], dtype = int)
# non_compliance_chain_3[:,8] = np.array(non_compliance_chain_3[:,8], dtype = int)
# non_compliance_chain_3[:,9] = np.array(non_compliance_chain_3[:,9], dtype = int)
# non_compliance_chain_3[:,10] = np.array(non_compliance_chain_3[:,10], dtype = int)

# non_compliance_chain_4[:,0] = np.array(non_compliance_chain_4[:,0], dtype = int)
# non_compliance_chain_4[:,8] = np.array(non_compliance_chain_4[:,8], dtype = int)
# non_compliance_chain_4[:,9] = np.array(non_compliance_chain_4[:,9], dtype = int)
# non_compliance_chain_4[:,10] = np.array(non_compliance_chain_4[:,10], dtype = int)
stochastic_chain_1[:,0]= np.array(stochastic_chain_1[:,0], dtype = int)
stochastic_chain_1[:,8]= np.array(stochastic_chain_1[:,8], dtype = int)

stochastic_chain_2[:,0]= np.array(stochastic_chain_2[:,0], dtype = int)
stochastic_chain_2[:,8]= np.array(stochastic_chain_2[:,8], dtype = int)

stochastic_chain_3[:,0]= np.array(stochastic_chain_3[:,0], dtype = int)
stochastic_chain_3[:,8]= np.array(stochastic_chain_3[:,8], dtype = int)

stochastic_chain_4[:,0]= np.array(stochastic_chain_4[:,0], dtype = int)
stochastic_chain_4[:,8]= np.array(stochastic_chain_4[:,8], dtype = int)

stochastic_sp_reparam_chain_1[:,0] = np.array(stochastic_sp_reparam_chain_1[:,0], dtype = int)
stochastic_sp_reparam_chain_1[:,8] = np.array(stochastic_sp_reparam_chain_1[:,8], dtype = int)

stochastic_sp_reparam_chain_2[:,0] = np.array(stochastic_sp_reparam_chain_2[:,0], dtype = int)
stochastic_sp_reparam_chain_2[:,8] = np.array(stochastic_sp_reparam_chain_2[:,8], dtype = int)

stochastic_sp_reparam_chain_3[:,0] = np.array(stochastic_sp_reparam_chain_3[:,0], dtype = int)
stochastic_sp_reparam_chain_3[:,8] = np.array(stochastic_sp_reparam_chain_3[:,8], dtype = int)

stochastic_sp_reparam_chain_1[:,0] = np.array(stochastic_sp_reparam_chain_4[:,0], dtype = int)
stochastic_sp_reparam_chain_1[:,8] = np.array(stochastic_sp_reparam_chain_4[:,8], dtype = int)

sp_reparam_chain_1[:,0] = np.array(sp_reparam_chain_1[:,0], dtype = int)
sp_reparam_chain_1[:,8] = np.array(sp_reparam_chain_1[:,8], dtype = int)

sp_reparam_chain_2[:,0] = np.array(sp_reparam_chain_2[:,0], dtype = int)
sp_reparam_chain_2[:,8] = np.array(sp_reparam_chain_2[:,8], dtype = int)

sp_reparam_chain_3[:,0] = np.array(sp_reparam_chain_3[:,0], dtype = int)
sp_reparam_chain_3[:,8] = np.array(sp_reparam_chain_3[:,8], dtype = int)


sp_reparam_chain_4[:,0] = np.array(sp_reparam_chain_4[:,0], dtype = int)
sp_reparam_chain_4[:,8] = np.array(sp_reparam_chain_4[:,8], dtype = int)

model_exper_no_uncertain_chain_1[:,0] = np.array(model_exper_no_uncertain_chain_1[:,0], dtype = int)
model_exper_no_uncertain_chain_1[:,8] = np.array(model_exper_no_uncertain_chain_1[:,8], dtype = int)

model_exper_no_uncertain_chain_2[:,0] = np.array(model_exper_no_uncertain_chain_2[:,0], dtype = int)
model_exper_no_uncertain_chain_2[:,8] = np.array(model_exper_no_uncertain_chain_2[:,8], dtype = int)

model_exper_no_uncertain_chain_3[:,0] = np.array(model_exper_no_uncertain_chain_3[:,0], dtype = int)
model_exper_no_uncertain_chain_3[:,8] = np.array(model_exper_no_uncertain_chain_3[:,8], dtype = int)

model_exper_no_uncertain_chain_4[:,0] = np.array(model_exper_no_uncertain_chain_4[:,0], dtype = int)
model_exper_no_uncertain_chain_4[:,8] = np.array(model_exper_no_uncertain_chain_4[:,8], dtype = int)

model_exper_uncertain_chain_1[:,0] = np.array(model_exper_uncertain_chain_1[:,0], dtype = int)
model_exper_uncertain_chain_1[:,8] = np.array(model_exper_uncertain_chain_1[:,8], dtype = int)

model_exper_uncertain_chain_2[:,0] = np.array(model_exper_uncertain_chain_2[:,0], dtype = int)
model_exper_uncertain_chain_2[:,8] = np.array(model_exper_uncertain_chain_2[:,8], dtype = int)

model_exper_uncertain_chain_3[:,0] = np.array(model_exper_uncertain_chain_3[:,0], dtype = int)
model_exper_uncertain_chain_3[:,8] = np.array(model_exper_uncertain_chain_3[:,8], dtype = int)

model_exper_uncertain_chain_4[:,0] = np.array(model_exper_uncertain_chain_4[:,0], dtype = int)
model_exper_uncertain_chain_4[:,8] = np.array(model_exper_uncertain_chain_4[:,8], dtype = int)

model_exper_rt_fixed_chain_1[:,0] = np.array(model_exper_rt_fixed_chain_1[:,0], dtype = int)
model_exper_rt_fixed_chain_1[:,7] = np.array(model_exper_rt_fixed_chain_1[:,7], dtype = int)

model_exper_rt_fixed_chain_2[:,0] = np.array(model_exper_rt_fixed_chain_2[:,0], dtype = int)
model_exper_rt_fixed_chain_2[:,7] = np.array(model_exper_rt_fixed_chain_2[:,7], dtype = int)

model_exper_rt_fixed_chain_3[:,0] = np.array(model_exper_rt_fixed_chain_3[:,0], dtype = int)
model_exper_rt_fixed_chain_3[:,7] = np.array(model_exper_rt_fixed_chain_3[:,7], dtype = int)

model_exper_rt_fixed_chain_4[:,0] = np.array(model_exper_rt_fixed_chain_4[:,0], dtype = int)
model_exper_rt_fixed_chain_4[:,7] = np.array(model_exper_rt_fixed_chain_4[:,7], dtype = int)


# model_exper_rt_fixed_community_params_fixed_chain_1[:,0] = np.array(model_exper_rt_fixed_community_params_fixed_chain_1[:,0], dtype = int)
# model_exper_rt_fixed_community_params_fixed_chain_1[:,7] = np.array(model_exper_rt_fixed_community_params_fixed_chain_1[:,7], dtype = int)

# model_exper_rt_fixed_community_params_fixed_chain_2[:,0] = np.array(model_exper_rt_fixed_community_params_fixed_chain_2[:,0], dtype = int)
# model_exper_rt_fixed_community_params_fixed_chain_2[:,8] = np.array(model_exper_rt_fixed_community_params_fixed_chain_2[:,8], dtype = int)

# model_exper_rt_fixed_community_params_fixed_chain_3[:,0] = np.array(model_exper_rt_fixed_community_params_fixed_chain_3[:,0], dtype = int)
# model_exper_rt_fixed_community_params_fixed_chain_3[:,8] = np.array(model_exper_rt_fixed_community_params_fixed_chain_3[:,8], dtype = int)

# model_exper_rt_fixed_community_params_fixed_chain_4[:,0] = np.array(model_exper_rt_fixed_community_params_fixed_chain_4[:,0], dtype = int)
# model_exper_rt_fixed_community_params_fixed_chain_4[:,8] = np.array(model_exper_rt_fixed_community_params_fixed_chain_4[:,8], dtype = int)

model_exper_adapted_LL_chain_1[:,0] = np.array(model_exper_adapted_LL_chain_1[:,0], dtype = int)
model_exper_adapted_LL_chain_1[:,8] = np.array(model_exper_adapted_LL_chain_1[:,8], dtype = int)

model_exper_adapted_LL_chain_2[:,0] = np.array(model_exper_adapted_LL_chain_2[:,0], dtype = int)
model_exper_adapted_LL_chain_2[:,8] = np.array(model_exper_adapted_LL_chain_2[:,8], dtype = int)

model_exper_adapted_LL_chain_3[:,0] = np.array(model_exper_adapted_LL_chain_3[:,0], dtype = int)
model_exper_adapted_LL_chain_3[:,8] = np.array(model_exper_adapted_LL_chain_3[:,8], dtype = int)

model_exper_adapted_LL_chain_4[:,0] = np.array(model_exper_adapted_LL_chain_4[:,0], dtype = int)
model_exper_adapted_LL_chain_4[:,8] = np.array(model_exper_adapted_LL_chain_4[:,8], dtype = int)



rit_data_adapted_LL_chain_1[:,0] = np.array(rit_data_adapted_LL_chain_1[:,0], dtype = int)
rit_data_adapted_LL_chain_1[:,8] = np.array(rit_data_adapted_LL_chain_1[:,8], dtype = int)

# rit_data_adapted_LL_chain_2 = pd.read_csv(r'', names = ['Iter', 'Frequency of ES', 'Time to Recovery', 'Percent Advancing', ' Rt', 'Symptom Case Fatality','Days to Incubation', 'Test Sensitivity', 'Test Specificity', 'New Infections per Shock'])
rit_data_adapted_LL_chain_3[:,0] = np.array(rit_data_adapted_LL_chain_3[:,0], dtype = int)
rit_data_adapted_LL_chain_3[:,8] = np.array(rit_data_adapted_LL_chain_3[:,8], dtype = int)

rit_data_adapted_LL_chain_4[:,0] = np.array(rit_data_adapted_LL_chain_4[:,0], dtype = int)
rit_data_adapted_LL_chain_4[:,8] = np.array(rit_data_adapted_LL_chain_4[:,8], dtype = int)


rit_data_adapted_LL_init_infect_chain_1[:,0] = np.array(rit_data_adapted_LL_init_infect_chain_1[:,0], dtype = int)
rit_data_adapted_LL_init_infect_chain_1[:,8] = np.array(rit_data_adapted_LL_init_infect_chain_1[:,8], dtype = int)

rit_data_adapted_LL_init_infect_chain_2[:,0] = np.array(rit_data_adapted_LL_init_infect_chain_2[:,0], dtype = int)
rit_data_adapted_LL_init_infect_chain_2[:,8] = np.array(rit_data_adapted_LL_init_infect_chain_2[:,8], dtype = int)

rit_data_adapted_LL_init_infect_chain_3[:,0] = np.array(rit_data_adapted_LL_init_infect_chain_3[:,0], dtype = int)
rit_data_adapted_LL_init_infect_chain_3[:,8] = np.array(rit_data_adapted_LL_init_infect_chain_3[:,8], dtype = int)

rit_data_adapted_LL_init_infect_chain_4[:,0] = np.array(rit_data_adapted_LL_init_infect_chain_4[:,0], dtype = int)
rit_data_adapted_LL_init_infect_chain_4[:,8] = np.array(rit_data_adapted_LL_init_infect_chain_4[:,8], dtype = int)

rit_data_adapted_LL_init_infect_stochastic_weight_chain_1[:,0] = np.array(rit_data_adapted_LL_init_infect_stochastic_weight_chain_1[:,0], dtype = int)
rit_data_adapted_LL_init_infect_stochastic_weight_chain_1[:,8] = np.array(rit_data_adapted_LL_init_infect_stochastic_weight_chain_1[:,8], dtype = int)

rit_data_adapted_LL_init_infect_stochastic_weight_chain_2[:,0] = np.array(rit_data_adapted_LL_init_infect_stochastic_weight_chain_2[:,0], dtype = int)
rit_data_adapted_LL_init_infect_stochastic_weight_chain_2[:,8] = np.array(rit_data_adapted_LL_init_infect_stochastic_weight_chain_2[:,8], dtype = int)

rit_data_adapted_LL_init_infect_stochastic_weight_chain_3[:,0] = np.array(rit_data_adapted_LL_init_infect_stochastic_weight_chain_3[:,0], dtype = int)
rit_data_adapted_LL_init_infect_stochastic_weight_chain_3[:,8] = np.array(rit_data_adapted_LL_init_infect_stochastic_weight_chain_3[:,8], dtype = int)

rit_data_adapted_LL_init_infect_stochastic_weight_chain_4[:,0] = np.array(rit_data_adapted_LL_init_infect_stochastic_weight_chain_4[:,0], dtype = int)
rit_data_adapted_LL_init_infect_stochastic_weight_chain_4[:,8] = np.array(rit_data_adapted_LL_init_infect_stochastic_weight_chain_4[:,8], dtype = int)


In [14]:
convergence_threshold_og_cal = 70000
convergence_threshold_init_cal = 90000
convergence_threshold_sp_reparam = 60000
convergence_threshold_stochastic_sp_reparam = 50000
convergence_threshold_stochastic = 90000

convergence_threshold_model_exper_uncertain = 50000
convergence_threshold_model_exper_no_uncertain = 50000

convergence_threshold_model_exper_rt_fixed_chain = 60000
convergence_threshold_model_exper_rt_community_transmission_fixed = 8000

convergence_threshold_model_exper_adapted_LL = 170000
convergence_threshold_adapted_LL = 160000

convergence_threshold_adapted_LL_init_infect = 160000

convergence_threshold_adapted_LL_stochastic_weight = 20000

## Burned in Samples
### OG Calibration

In [15]:
og_calibration_burn_in_1 = og_calibration_chain_1[convergence_threshold_og_cal:, :]
og_calibration_burn_in_2 = og_calibration_chain_2[convergence_threshold_og_cal:, :]
og_calibration_burn_in_3 = og_calibration_chain_3[convergence_threshold_og_cal:, :]
og_calibration_burn_in_4 = og_calibration_chain_4[convergence_threshold_og_cal:, :]

np.shape(og_calibration_burn_in_1)

(30000, 9)

### Initial Infected and Adjusted Testing Term

In [16]:
initi_infect_adjust_test_terms_chain_1_burn_in = initi_infect_adjust_test_terms_chain_1 [convergence_threshold_init_cal:, :]
initi_infect_adjust_test_terms_chain_2_burn_in = initi_infect_adjust_test_terms_chain_2[convergence_threshold_init_cal:, :]
initi_infect_adjust_test_terms_chain_3_burn_in = initi_infect_adjust_test_terms_chain_3[convergence_threshold_init_cal:, :]
initi_infect_adjust_test_terms_chain_4_burn_in = initi_infect_adjust_test_terms_chain_4[convergence_threshold_init_cal:, :]

np.shape(initi_infect_adjust_test_terms_chain_2_burn_in)

(10000, 10)

### Reparam Sp

In [17]:
sp_reparam_chain_1_burn_in = sp_reparam_chain_1 [convergence_threshold_sp_reparam:, :]
sp_reparam_chain_2_burn_in = sp_reparam_chain_2[convergence_threshold_sp_reparam:, :]
sp_reparam_chain_3_burn_in = sp_reparam_chain_3 [convergence_threshold_sp_reparam:, :]
sp_reparam_chain_4_burn_in = sp_reparam_chain_4[convergence_threshold_sp_reparam:, :]

np.shape(sp_reparam_chain_1_burn_in)

(40000, 9)

### Stochasticity

In [18]:
stochastic_chain_1_burn_in = stochastic_chain_1[convergence_threshold_stochastic:,:]
stochastic_chain_2_burn_in = stochastic_chain_2[convergence_threshold_stochastic:,:]
stochastic_chain_3_burn_in = stochastic_chain_3[convergence_threshold_stochastic:,:]
stochastic_chain_4_burn_in = stochastic_chain_4[convergence_threshold_stochastic:,:]

np.shape(stochastic_chain_1_burn_in)

(10000, 9)

### Stochastic and Sp reparam

In [19]:
stochastic_sp_reparam_chain_1_burn_in = stochastic_sp_reparam_chain_1[convergence_threshold_stochastic_sp_reparam:,:]
stochastic_sp_reparam_chain_2_burn_in = stochastic_sp_reparam_chain_2[convergence_threshold_stochastic_sp_reparam:,:]
stochastic_sp_reparam_chain_3_burn_in = stochastic_sp_reparam_chain_3[convergence_threshold_stochastic_sp_reparam:,:]
stochastic_sp_reparam_chain_4_burn_in = stochastic_sp_reparam_chain_4[convergence_threshold_stochastic_sp_reparam:,:]


np.shape(stochastic_sp_reparam_chain_1_burn_in)

(50000, 9)

### Model Experiment NO Uncertainty

In [20]:
model_exper_no_uncertain_chain_1_burn_in = model_exper_no_uncertain_chain_1[convergence_threshold_model_exper_no_uncertain:,:]
model_exper_no_uncertain_chain_2_burn_in = model_exper_no_uncertain_chain_2[convergence_threshold_model_exper_no_uncertain:,:]
model_exper_no_uncertain_chain_3_burn_in = model_exper_no_uncertain_chain_3[convergence_threshold_model_exper_no_uncertain:,:]
model_exper_no_uncertain_chain_4_burn_in = model_exper_no_uncertain_chain_4[convergence_threshold_model_exper_no_uncertain:,:]

np.shape(model_exper_no_uncertain_chain_1_burn_in)

(50000, 9)

### Model Experiment w/ Uncertainty

In [21]:
model_exper_uncertain_chain_1_burn_in = model_exper_uncertain_chain_1[convergence_threshold_model_exper_uncertain:,:]
model_exper_uncertain_chain_2_burn_in = model_exper_uncertain_chain_2[convergence_threshold_model_exper_uncertain:,:]
model_exper_uncertain_chain_3_burn_in = model_exper_uncertain_chain_3[convergence_threshold_model_exper_uncertain:,:]
model_exper_uncertain_chain_4_burn_in = model_exper_uncertain_chain_4[convergence_threshold_model_exper_uncertain:,:]

np.shape(model_exper_uncertain_chain_1_burn_in)

(50000, 9)

### Model Experiment w/ Uncertainty Rt Fixed

In [22]:
model_exper_rt_fixed_chain_1_burn_in = model_exper_rt_fixed_chain_1[convergence_threshold_model_exper_rt_fixed_chain:,:]
model_exper_rt_fixed_chain_2_burn_in = model_exper_rt_fixed_chain_2[convergence_threshold_model_exper_rt_fixed_chain:,:]
model_exper_rt_fixed_chain_3_burn_in = model_exper_rt_fixed_chain_3[convergence_threshold_model_exper_rt_fixed_chain:,:]
model_exper_rt_fixed_chain_4_burn_in = model_exper_rt_fixed_chain_4[convergence_threshold_model_exper_rt_fixed_chain:,:]

np.shape(model_exper_rt_fixed_chain_1_burn_in)

(90000, 8)

### Model Experiment w/ Uncertainty Rt and Community Transmission Parameters Fixed

In [23]:
model_exper_rt_fixed_community_params_fixed_chain_1_burn_in = model_exper_rt_fixed_community_params_fixed_chain_1[convergence_threshold_model_exper_rt_community_transmission_fixed:,:]
model_exper_rt_fixed_community_params_fixed_chain_2_burn_in = model_exper_rt_fixed_community_params_fixed_chain_2[convergence_threshold_model_exper_rt_community_transmission_fixed:,:]
model_exper_rt_fixed_community_params_fixed_chain_3_burn_in = model_exper_rt_fixed_community_params_fixed_chain_3[convergence_threshold_model_exper_rt_community_transmission_fixed:,:]
model_exper_rt_fixed_community_params_fixed_chain_4_burn_in = model_exper_rt_fixed_community_params_fixed_chain_4[convergence_threshold_model_exper_rt_community_transmission_fixed:,:]

np.shape(model_exper_rt_fixed_community_params_fixed_chain_1_burn_in)

(92000, 6)

### Model Experiment w/ adapted LL

In [24]:
model_exper_adapted_LL_chain_1_burn_in = model_exper_adapted_LL_chain_1[convergence_threshold_model_exper_adapted_LL:,:]
model_exper_adapted_LL_chain_2_burn_in = model_exper_adapted_LL_chain_2[convergence_threshold_model_exper_adapted_LL:,:]
model_exper_adapted_LL_chain_3_burn_in = model_exper_adapted_LL_chain_3[convergence_threshold_model_exper_adapted_LL:,:]
model_exper_adapted_LL_chain_4_burn_in = model_exper_adapted_LL_chain_4[convergence_threshold_model_exper_adapted_LL:,:]


np.shape(model_exper_adapted_LL_chain_1_burn_in)

(30000, 9)

### RIT Data w/ adapted LL

In [25]:
rit_data_adapted_LL_chain_1_burn_in = rit_data_adapted_LL_chain_1[convergence_threshold_adapted_LL:,:]
# rit_data_adapted_LL_chain_2_burn_in = rit_data_adapted_LL_chain_1[convergence_threshold_adapted_LL:,:]model_exper_adapted_LL_chain_3_burn_in = model_exper_adapted_LL_chain_3[convergence_threshold_model_exper_adapted_LL:,:]
rit_data_adapted_LL_chain_3_burn_in = rit_data_adapted_LL_chain_3[convergence_threshold_adapted_LL:,:]
rit_data_adapted_LL_chain_4_burn_in = rit_data_adapted_LL_chain_4[convergence_threshold_adapted_LL:,:]

np.shape(rit_data_adapted_LL_chain_1_burn_in)

(140000, 9)

In [26]:
rit_data_adapted_LL_chain_4_burn_in-rit_data_adapted_LL_chain_1_burn_in

array([[ 1.00000000e+00, -4.61325187e-01,  1.14574112e+00, ...,
         1.29149130e-01,  4.86771890e-03,  3.00000000e+00],
       [ 1.00000000e+00, -4.61325187e-01,  1.14574112e+00, ...,
         1.29149130e-01,  4.86771890e-03,  3.00000000e+00],
       [ 1.00000000e+00, -4.61325187e-01,  1.14574112e+00, ...,
         1.29149130e-01,  4.86771890e-03,  3.00000000e+00],
       ...,
       [ 3.00000000e+00, -3.61481318e-02,  3.14092881e+00, ...,
        -1.30504254e-02, -2.67561977e-03,  8.00000000e+00],
       [ 3.00000000e+00, -3.61481318e-02,  3.14092881e+00, ...,
        -1.30504254e-02, -2.67561977e-03,  8.00000000e+00],
       [ 3.00000000e+00, -3.61481318e-02,  3.14092881e+00, ...,
        -1.30504254e-02, -2.67561977e-03,  8.00000000e+00]])

### RIT Data w/ adapted LL and Initial Infected

In [27]:
rit_data_adapted_LL_init_infect_chain_1_burn_in = rit_data_adapted_LL_init_infect_chain_1[convergence_threshold_adapted_LL_init_infect:,:]
rit_data_adapted_LL_init_infect_chain_2_burn_in = rit_data_adapted_LL_init_infect_chain_2[convergence_threshold_adapted_LL_init_infect:,:]
rit_data_adapted_LL_init_infect_chain_3_burn_in = rit_data_adapted_LL_init_infect_chain_3[convergence_threshold_adapted_LL_init_infect:,:]
rit_data_adapted_LL_init_infect_chain_4_burn_in = rit_data_adapted_LL_init_infect_chain_4[convergence_threshold_adapted_LL_init_infect:,:]

np.shape(rit_data_adapted_LL_init_infect_chain_1_burn_in )

(440000, 10)

### RIT Data w/ adapted LL Initial infect, initial iso, stochastic test, weighted LL

In [29]:
rit_data_adapted_LL_init_infect_stochastic_weight_chain_1_burn_in = rit_data_adapted_LL_init_infect_stochastic_weight_chain_1[convergence_threshold_adapted_LL_stochastic_weight:,:]
rit_data_adapted_LL_init_infect_stochastic_weight_chain_2_burn_in = rit_data_adapted_LL_init_infect_stochastic_weight_chain_2[convergence_threshold_adapted_LL_stochastic_weight:,:]
rit_data_adapted_LL_init_infect_stochastic_weight_chain_3_burn_in = rit_data_adapted_LL_init_infect_stochastic_weight_chain_3[convergence_threshold_adapted_LL_stochastic_weight:,:]
rit_data_adapted_LL_init_infect_stochastic_weight_chain_4_burn_in = rit_data_adapted_LL_init_infect_stochastic_weight_chain_4[convergence_threshold_adapted_LL_stochastic_weight:,:]

### Format Burned In States as a Tensor
#### OG Calibration


In [34]:
states_og_cal_1 = tf.convert_to_tensor(og_calibration_burn_in_1)
states_og_cal_2 = tf.convert_to_tensor(og_calibration_burn_in_2)
states_og_cal_3 = tf.convert_to_tensor(og_calibration_burn_in_3)
states_og_cal_4 = tf.convert_to_tensor(og_calibration_burn_in_4)

In [35]:
states_og_cal_1 

<tf.Tensor: shape=(30000, 9), dtype=float64, numpy=
array([[ 5.        , 14.18324262, 27.67164469, ...,  0.76391017,
         0.99898303,  8.        ],
       [ 5.        , 14.18324262, 27.67164469, ...,  0.76391017,
         0.99898303,  8.        ],
       [ 4.        , 16.73751851, 33.49892791, ...,  0.77281602,
         0.99926775, 10.        ],
       ...,
       [ 6.        , 15.57948649, 24.04413283, ...,  0.79610014,
         0.99792392, 11.        ],
       [ 6.        , 15.57948649, 24.04413283, ...,  0.79610014,
         0.99792392, 11.        ],
       [ 7.        , 13.77061373, 21.41370521, ...,  0.78924635,
         0.99699309, 13.        ]])>

In [36]:
ess_og_cal_1 = tfp.mcmc.effective_sample_size(states_og_cal_1)
print(ess_og_cal_1)
ess_og_cal_2 = tfp.mcmc.effective_sample_size(states_og_cal_2)
print(ess_og_cal_2)
ess_og_cal_3 = tfp.mcmc.effective_sample_size(states_og_cal_3)
print(ess_og_cal_3)
ess_og_cal_4 = tfp.mcmc.effective_sample_size(states_og_cal_4)
print(ess_og_cal_4)

tf.Tensor(
[ 725.59140784  646.21908076  462.03338379  490.80319845  651.52007753
  588.96071054  457.51953289 1134.54846115  463.76639972], shape=(9,), dtype=float64)
tf.Tensor(
[547.96553091 456.88269922 428.80471259 470.7228077  721.07829879
 474.19174434 542.93281809 701.27622296 729.44568736], shape=(9,), dtype=float64)
tf.Tensor(
[332.95272726 315.00576417 333.03594689 293.99853184 405.3105894
 397.1725097  562.99196712 379.44417557 645.92316163], shape=(9,), dtype=float64)
tf.Tensor(
[560.93550576 450.11124078 529.96979194 534.244163   450.21887234
 522.93526802 519.40781471 635.60633017 483.73961637], shape=(9,), dtype=float64)


In [37]:
print('Min 1:', min(ess_og_cal_1))
print('Min 2:', min(ess_og_cal_2))
print('Min 3:', min(ess_og_cal_3))
print('Min 4:', min(ess_og_cal_4))

print('Min all:', min((min(ess_og_cal_1),min(ess_og_cal_2), min(ess_og_cal_3), min(ess_og_cal_4))))

Min 1: tf.Tensor(457.5195328923628, shape=(), dtype=float64)
Min 2: tf.Tensor(428.80471258751294, shape=(), dtype=float64)
Min 3: tf.Tensor(293.9985318426003, shape=(), dtype=float64)
Min 4: tf.Tensor(450.11124078075136, shape=(), dtype=float64)
Min all: tf.Tensor(293.9985318426003, shape=(), dtype=float64)


#### Initial Infect Adjusted Testing Term

In [38]:
states_init_cal = tf.convert_to_tensor(initi_infect_adjust_test_terms_burn_in)

NameError: name 'initi_infect_adjust_test_terms_burn_in' is not defined

In [39]:
states_init_cal

NameError: name 'states_init_cal' is not defined

In [40]:
ess_init_cal = tfp.mcmc.effective_sample_size(states_init_cal)
print(ess_init_cal)

NameError: name 'states_init_cal' is not defined

In [41]:
print('Max:', max(ess_init_cal))
print('Min:', min(ess_init_cal))

NameError: name 'ess_init_cal' is not defined

#### Test Specificity Reparameter

In [42]:
states_sp_reparam_cal_1 = tf.convert_to_tensor(sp_reparam_chain_1_burn_in)
states_sp_reparam_cal_2 = tf.convert_to_tensor(sp_reparam_chain_2_burn_in)
states_sp_reparam_cal_3 = tf.convert_to_tensor(sp_reparam_chain_3_burn_in)
states_sp_reparam_cal_4 = tf.convert_to_tensor(sp_reparam_chain_4_burn_in)

In [43]:
states_sp_reparam_cal_1

<tf.Tensor: shape=(40000, 9), dtype=float64, numpy=
array([[ 6.        , 14.85617421, 11.18452068, ...,  0.8025827 ,
         1.24135239, 12.        ],
       [ 6.        , 14.85617421, 11.18452068, ...,  0.8025827 ,
         1.24135239, 12.        ],
       [ 6.        , 14.85617421, 11.18452068, ...,  0.8025827 ,
         1.24135239, 12.        ],
       ...,
       [ 8.        , 12.74244405, 15.28584111, ...,  0.80001603,
         1.24346768, 12.        ],
       [ 8.        , 12.74244405, 15.28584111, ...,  0.80001603,
         1.24346768, 12.        ],
       [ 8.        , 12.74244405, 15.28584111, ...,  0.80001603,
         1.24346768, 12.        ]])>

In [44]:
ess_sp_reparam_cal_1 = tfp.mcmc.effective_sample_size(states_sp_reparam_cal_1)
print(ess_sp_reparam_cal_1)
ess_sp_reparam_cal_2 = tfp.mcmc.effective_sample_size(states_sp_reparam_cal_2)
print(ess_sp_reparam_cal_2)
ess_sp_reparam_cal_3 = tfp.mcmc.effective_sample_size(states_sp_reparam_cal_3)
print(ess_sp_reparam_cal_3)
ess_sp_reparam_cal_4 = tfp.mcmc.effective_sample_size(states_sp_reparam_cal_4)
print(ess_sp_reparam_cal_4)

tf.Tensor(
[ 477.31577348  526.86681     606.44883438  556.29066762  525.73437689
  463.11982607  581.30122662  565.5379492  1031.89926393], shape=(9,), dtype=float64)
tf.Tensor(
[ 872.43568806 1093.90058739 1014.21375507  577.69864865  967.54471312
  935.98543385  700.12330913  958.9761635   585.21196839], shape=(9,), dtype=float64)
tf.Tensor(
[1029.74645284  843.16476797  936.95971463  911.27949973 1007.69736636
  976.98046222  832.86626358  736.60919344  701.76439898], shape=(9,), dtype=float64)
tf.Tensor(
[1003.87224393  919.66597284  984.35028671  844.34283335  942.730305
  842.90230919  997.37764305  858.04344636  783.77504167], shape=(9,), dtype=float64)


In [45]:
print('Min 1:', min(ess_sp_reparam_cal_1))
print('Min 2:', min(ess_sp_reparam_cal_2))
print('Min 3:', min(ess_sp_reparam_cal_3))
print('Min 4:', min(ess_sp_reparam_cal_4))

print('Min all:', min((min(ess_sp_reparam_cal_1),min(ess_sp_reparam_cal_2), min(ess_sp_reparam_cal_3), min(ess_sp_reparam_cal_4))))

Min 1: tf.Tensor(463.11982607380327, shape=(), dtype=float64)
Min 2: tf.Tensor(577.6986486516819, shape=(), dtype=float64)
Min 3: tf.Tensor(701.7643989847925, shape=(), dtype=float64)
Min 4: tf.Tensor(783.7750416715699, shape=(), dtype=float64)
Min all: tf.Tensor(463.11982607380327, shape=(), dtype=float64)


#### Stochasticity

In [46]:
states_stochastic_cal_1 = tf.convert_to_tensor(stochastic_chain_1_burn_in)
states_stochastic_cal_2 = tf.convert_to_tensor(stochastic_chain_2_burn_in)
states_stochastic_cal_3 = tf.convert_to_tensor(stochastic_chain_3_burn_in)
states_stochastic_cal_4 = tf.convert_to_tensor(stochastic_chain_4_burn_in)

In [47]:
states_stochastic_cal_1

<tf.Tensor: shape=(10000, 9), dtype=float64, numpy=
array([[ 9.        , 14.72545377, 38.99415577, ...,  0.85153948,
         0.99819364, 19.        ],
       [ 9.        , 14.72545377, 38.99415577, ...,  0.85153948,
         0.99819364, 19.        ],
       [ 9.        , 14.72545377, 38.99415577, ...,  0.85153948,
         0.99819364, 19.        ],
       ...,
       [ 7.        , 18.51653135, 23.58411289, ...,  0.83651457,
         0.99839923, 12.        ],
       [ 7.        , 18.51653135, 23.58411289, ...,  0.83651457,
         0.99839923, 12.        ],
       [ 7.        , 18.51653135, 23.58411289, ...,  0.83651457,
         0.99839923, 12.        ]])>

In [48]:
ess_stochastic_cal_1 = tfp.mcmc.effective_sample_size(states_stochastic_cal_1)
print(ess_stochastic_cal_1)
ess_stochastic_cal_2 = tfp.mcmc.effective_sample_size(states_stochastic_cal_2)
print(ess_stochastic_cal_2)
ess_stochastic_cal_3 = tfp.mcmc.effective_sample_size(states_stochastic_cal_3)
print(ess_stochastic_cal_3)
ess_stochastic_cal_4 = tfp.mcmc.effective_sample_size(states_stochastic_cal_4)
print(ess_stochastic_cal_4)

tf.Tensor(
[156.28106213 100.35162392 131.28632768 105.93398153 117.29086836
 133.07967907 122.40666606 144.21899928 204.92029292], shape=(9,), dtype=float64)
tf.Tensor(
[209.35005443 150.15976133 260.31123943 194.40313854 225.6971988
 229.4443526  229.08958545 277.45877623 213.527967  ], shape=(9,), dtype=float64)
tf.Tensor(
[111.66968044 212.78702146 223.11394814 174.2909858  124.59916917
 169.8918176  159.939996   369.52080215 197.97707884], shape=(9,), dtype=float64)
tf.Tensor(
[194.73591089 214.90640009 227.88725422  94.77700312 206.55235064
 221.77749182 223.41889337 163.86956891 220.00794054], shape=(9,), dtype=float64)


In [49]:
print('Min 1:', min(ess_stochastic_cal_1))
print('Min 2:', min(ess_stochastic_cal_2))
print('Min 3:', min(ess_stochastic_cal_3))
print('Min 4:', min(ess_stochastic_cal_4))

print('Min All:', min((min(ess_stochastic_cal_1), min(ess_stochastic_cal_2), min(ess_stochastic_cal_3), min(ess_stochastic_cal_4))))

Min 1: tf.Tensor(100.3516239157502, shape=(), dtype=float64)
Min 2: tf.Tensor(150.15976132697645, shape=(), dtype=float64)
Min 3: tf.Tensor(111.66968044201191, shape=(), dtype=float64)
Min 4: tf.Tensor(94.77700312408567, shape=(), dtype=float64)
Min All: tf.Tensor(94.77700312408567, shape=(), dtype=float64)


#### Stochasticity and Sp Reparam

In [50]:
states_stoch_sp_reparam_cal = tf.convert_to_tensor(stochastic_sp_reparam_burn_in)

NameError: name 'stochastic_sp_reparam_burn_in' is not defined

In [51]:
states_stoch_sp_reparam_cal

NameError: name 'states_stoch_sp_reparam_cal' is not defined

In [52]:
ess_stoch_sp_reparam_cal = tfp.mcmc.effective_sample_size(states_stoch_sp_reparam_cal)
print(ess_stoch_sp_reparam_cal)

NameError: name 'states_stoch_sp_reparam_cal' is not defined

In [53]:
print('Max:', max(ess_stoch_sp_reparam_cal))
print('Min:', min(ess_stoch_sp_reparam_cal))

NameError: name 'ess_stoch_sp_reparam_cal' is not defined

#### Model Exper No Uncertainty

In [54]:
states_model_exper_no_uncertain_cal_1 = tf.convert_to_tensor(model_exper_no_uncertain_chain_1_burn_in)
states_model_exper_no_uncertain_cal_2 = tf.convert_to_tensor(model_exper_no_uncertain_chain_2_burn_in)
states_model_exper_no_uncertain_cal_3 = tf.convert_to_tensor(model_exper_no_uncertain_chain_3_burn_in)
states_model_exper_no_uncertain_cal_4 = tf.convert_to_tensor(model_exper_no_uncertain_chain_4_burn_in)

In [55]:
states_model_exper_no_uncertain_cal_1

<tf.Tensor: shape=(50000, 9), dtype=float64, numpy=
array([[ 4.        , 13.32178481, 20.07521567, ...,  0.81902131,
         1.23856225, 14.        ],
       [ 4.        , 13.32178481, 20.07521567, ...,  0.81902131,
         1.23856225, 14.        ],
       [ 5.        , 15.33159052, 18.81357635, ...,  0.83336404,
         1.23541359, 16.        ],
       ...,
       [ 6.        , 15.53501835, 39.58686403, ...,  0.71933883,
         1.23832898, 14.        ],
       [ 6.        , 15.53501835, 39.58686403, ...,  0.71933883,
         1.23832898, 14.        ],
       [ 6.        , 15.53501835, 39.58686403, ...,  0.71933883,
         1.23832898, 14.        ]])>

In [56]:
ess_model_exper_no_uncertain_cal_1 = tfp.mcmc.effective_sample_size(states_model_exper_no_uncertain_cal_1)
print(ess_model_exper_no_uncertain_cal_1)
ess_model_exper_no_uncertain_cal_2 = tfp.mcmc.effective_sample_size(states_model_exper_no_uncertain_cal_2)
print(ess_model_exper_no_uncertain_cal_2)
ess_model_exper_no_uncertain_cal_3 = tfp.mcmc.effective_sample_size(states_model_exper_no_uncertain_cal_3)
print(ess_model_exper_no_uncertain_cal_3)
ess_model_exper_no_uncertain_cal_4 = tfp.mcmc.effective_sample_size(states_model_exper_no_uncertain_cal_4)
print(ess_model_exper_no_uncertain_cal_4)

tf.Tensor(
[1182.37195926  971.19064635 1027.78665044 1146.37508257 1220.03711931
  928.41831314 1212.38424208  989.9139789  1026.80968212], shape=(9,), dtype=float64)
tf.Tensor(
[ 936.31643643  714.38217702  814.61093903  766.71376588  757.85321638
  807.603668    773.84108707  818.98526856 1343.85881464], shape=(9,), dtype=float64)
tf.Tensor(
[1185.19640549 1174.66512352 1209.48011656 1208.16158486 1158.75686994
 1370.65678129 1251.26166455 1302.68889167 1032.33587221], shape=(9,), dtype=float64)
tf.Tensor(
[1148.18778871 1243.16124999 1322.14527107 1010.36831218 1153.35283708
 1148.3401805  1316.80671862 1258.80146212  861.05934398], shape=(9,), dtype=float64)


In [57]:
print('Min 1:', min(ess_model_exper_no_uncertain_cal_1))
print('Min 2:', min(ess_model_exper_no_uncertain_cal_2))
print('Min 3:', min(ess_model_exper_no_uncertain_cal_3))
print('Min 4:', min(ess_model_exper_no_uncertain_cal_4))

print('Min all:', min((min(ess_model_exper_no_uncertain_cal_1), min(ess_model_exper_no_uncertain_cal_2), min(ess_model_exper_no_uncertain_cal_3), min(ess_model_exper_no_uncertain_cal_4))))

Min 1: tf.Tensor(928.41831314197, shape=(), dtype=float64)
Min 2: tf.Tensor(714.3821770176911, shape=(), dtype=float64)
Min 3: tf.Tensor(1032.3358722087507, shape=(), dtype=float64)
Min 4: tf.Tensor(861.0593439769353, shape=(), dtype=float64)
Min all: tf.Tensor(714.3821770176911, shape=(), dtype=float64)


#### Model Exper With Uncertainty

In [58]:
states_model_exper_uncertain_cal_1 = tf.convert_to_tensor(model_exper_uncertain_chain_1_burn_in)
states_model_exper_uncertain_cal_2 = tf.convert_to_tensor(model_exper_uncertain_chain_2_burn_in)
states_model_exper_uncertain_cal_3 = tf.convert_to_tensor(model_exper_uncertain_chain_3_burn_in)
states_model_exper_uncertain_cal_4 = tf.convert_to_tensor(model_exper_uncertain_chain_4_burn_in)


In [59]:
states_model_exper_uncertain_cal_1 

<tf.Tensor: shape=(50000, 9), dtype=float64, numpy=
array([[ 8.        , 14.53687906, 18.18158419, ...,  0.72320445,
         1.22309479, 19.        ],
       [ 8.        , 14.53687906, 18.18158419, ...,  0.72320445,
         1.22309479, 19.        ],
       [ 8.        , 14.53687906, 18.18158419, ...,  0.72320445,
         1.22309479, 19.        ],
       ...,
       [ 7.        , 16.92108473, 17.57931702, ...,  0.75624958,
         1.23153156, 13.        ],
       [ 9.        , 14.82295548, 21.62588911, ...,  0.78256672,
         1.22638031,  9.        ],
       [ 9.        , 14.82295548, 21.62588911, ...,  0.78256672,
         1.22638031,  9.        ]])>

In [60]:
ess_model_exper_uncertain_cal_1 = tfp.mcmc.effective_sample_size(states_model_exper_uncertain_cal_1 )
print(ess_model_exper_uncertain_cal_1)
ess_model_exper_uncertain_cal_2 = tfp.mcmc.effective_sample_size(states_model_exper_uncertain_cal_2 )
print(ess_model_exper_uncertain_cal_2)
ess_model_exper_uncertain_cal_3 = tfp.mcmc.effective_sample_size(states_model_exper_uncertain_cal_3 )
print(ess_model_exper_uncertain_cal_3)
ess_model_exper_uncertain_cal_4 = tfp.mcmc.effective_sample_size(states_model_exper_uncertain_cal_4 )
print(ess_model_exper_uncertain_cal_4)

tf.Tensor(
[1115.9168921  1073.42045651 1267.25159773 1168.48404098 1069.31564876
 1094.85737893 1199.53758928  998.77846421 1085.19481242], shape=(9,), dtype=float64)
tf.Tensor(
[ 879.97146574 1259.38885236 1119.85182213 1155.44003187 1117.13379549
 1191.37391707 1306.54266397  970.04473255  922.76905357], shape=(9,), dtype=float64)
tf.Tensor(
[ 975.08443101 1129.58358321 1157.83410785  903.4559641  1134.37663933
  946.40596142 1289.42699388  954.33432325  917.98042291], shape=(9,), dtype=float64)
tf.Tensor(
[ 571.85911155  795.81921098  788.18140155  773.53956656  694.98578688
  604.3972099   904.08863788  774.25657874 1238.91216281], shape=(9,), dtype=float64)


In [61]:
print('Min 1:', min(ess_model_exper_uncertain_cal_1))
print('Min 2:', min(ess_model_exper_uncertain_cal_2))
print('Min 3:', min(ess_model_exper_uncertain_cal_3))
print('Min 4:', min(ess_model_exper_uncertain_cal_4))

print('Min all:', min((min(ess_model_exper_uncertain_cal_1), min(ess_model_exper_uncertain_cal_2), min(ess_model_exper_uncertain_cal_3), min(ess_model_exper_uncertain_cal_4))))

Min 1: tf.Tensor(998.778464214703, shape=(), dtype=float64)
Min 2: tf.Tensor(879.9714657381734, shape=(), dtype=float64)
Min 3: tf.Tensor(903.4559641018307, shape=(), dtype=float64)
Min 4: tf.Tensor(571.8591115509911, shape=(), dtype=float64)
Min all: tf.Tensor(571.8591115509911, shape=(), dtype=float64)


#### Model Exper w/ Uncertainty Fixed Rt

In [62]:
states_model_exper_fixed_rt_cal_1 = tf.convert_to_tensor(model_exper_rt_fixed_chain_1_burn_in)
states_model_exper_fixed_rt_cal_2 = tf.convert_to_tensor(model_exper_rt_fixed_chain_2_burn_in)
states_model_exper_fixed_rt_cal_3 = tf.convert_to_tensor(model_exper_rt_fixed_chain_3_burn_in)
states_model_exper_fixed_rt_cal_4 = tf.convert_to_tensor(model_exper_rt_fixed_chain_4_burn_in)

In [63]:
states_model_exper_fixed_rt_cal_1

<tf.Tensor: shape=(90000, 8), dtype=float64, numpy=
array([[ 7.        , 19.41003288, 30.70503955, ...,  0.85128525,
         1.23275024, 25.        ],
       [ 7.        , 19.41003288, 30.70503955, ...,  0.85128525,
         1.23275024, 25.        ],
       [ 7.        , 19.41003288, 30.70503955, ...,  0.85128525,
         1.23275024, 25.        ],
       ...,
       [ 7.        , 14.93193587, 35.21124391, ...,  0.74619954,
         1.24186673, 22.        ],
       [ 8.        , 11.81045379, 48.26674784, ...,  0.75955715,
         1.2375112 , 21.        ],
       [ 8.        , 11.81045379, 48.26674784, ...,  0.75955715,
         1.2375112 , 21.        ]])>

In [64]:
ess_model_exper_fixed_rt_cal_1 = tfp.mcmc.effective_sample_size(states_model_exper_fixed_rt_cal_1)
print(ess_model_exper_fixed_rt_cal_1)
ess_model_exper_fixed_rt_cal_2 = tfp.mcmc.effective_sample_size(states_model_exper_fixed_rt_cal_2)
print(ess_model_exper_fixed_rt_cal_2)
ess_model_exper_fixed_rt_cal_3 = tfp.mcmc.effective_sample_size(states_model_exper_fixed_rt_cal_3)
print(ess_model_exper_fixed_rt_cal_3)
ess_model_exper_fixed_rt_cal_4 = tfp.mcmc.effective_sample_size(states_model_exper_fixed_rt_cal_4)
print(ess_model_exper_fixed_rt_cal_4)

tf.Tensor(
[2296.11588943 2338.99344898 2522.88232967 2329.44400301 2287.07981346
 2547.67799579 2520.38357086 2372.15645096], shape=(8,), dtype=float64)
tf.Tensor(
[2379.9492931  2290.70668512 2359.75860714 2480.61784083 2342.76149815
 2554.38601958 2488.27715773 2067.35693235], shape=(8,), dtype=float64)
tf.Tensor(
[2491.64956217 2439.08023215 2022.6235267  2520.3825136  1945.90633368
 2277.39937891 2729.60858537 2234.43409211], shape=(8,), dtype=float64)
tf.Tensor(
[1740.58738685 1915.43507569 2019.0254701  2072.08247395 1607.64509842
 1894.86547372 2042.93976206 3328.20801559], shape=(8,), dtype=float64)


In [65]:
print('Min 1:', min(ess_model_exper_fixed_rt_cal_1))
print('Min 2:', min(ess_model_exper_fixed_rt_cal_2))
print('Min 3:', min(ess_model_exper_fixed_rt_cal_3))
print('Min 4:', min(ess_model_exper_fixed_rt_cal_4))

print('Min all', min((min(ess_model_exper_fixed_rt_cal_1), min(ess_model_exper_fixed_rt_cal_2), min(ess_model_exper_fixed_rt_cal_3), min(ess_model_exper_fixed_rt_cal_4))))

Min 1: tf.Tensor(2287.079813458197, shape=(), dtype=float64)
Min 2: tf.Tensor(2067.3569323507168, shape=(), dtype=float64)
Min 3: tf.Tensor(1945.9063336771053, shape=(), dtype=float64)
Min 4: tf.Tensor(1607.6450984201074, shape=(), dtype=float64)
Min all tf.Tensor(1607.6450984201074, shape=(), dtype=float64)


#### Model Exper w/ Uncertainty Fixed Rt Fixed Community Transmission Parameters

In [66]:
states_model_exper_fixed_rt_community_params_cal_1 = tf.convert_to_tensor(model_exper_rt_fixed_community_params_fixed_chain_1_burn_in)
states_model_exper_fixed_rt_community_params_cal_2 = tf.convert_to_tensor(model_exper_rt_fixed_community_params_fixed_chain_2_burn_in)
states_model_exper_fixed_rt_community_params_cal_3 = tf.convert_to_tensor(model_exper_rt_fixed_community_params_fixed_chain_3_burn_in)
states_model_exper_fixed_rt_community_params_cal_4 = tf.convert_to_tensor(model_exper_rt_fixed_community_params_fixed_chain_4_burn_in)


In [67]:
states_model_exper_fixed_rt_community_params_cal_1 

<tf.Tensor: shape=(92000, 6), dtype=float64, numpy=
array([[1.23139360e+01, 3.32445945e+01, 4.20467414e-03, 5.57926154e+00,
        7.92076353e-01, 1.21830296e+00],
       [1.23139360e+01, 3.32445945e+01, 4.20467414e-03, 5.57926154e+00,
        7.92076353e-01, 1.21830296e+00],
       [1.23139360e+01, 3.32445945e+01, 4.20467414e-03, 5.57926154e+00,
        7.92076353e-01, 1.21830296e+00],
       ...,
       [1.94215894e+01, 3.13888651e+01, 7.15042384e-03, 7.79300062e+00,
        7.81055000e-01, 1.22449088e+00],
       [1.94215894e+01, 3.13888651e+01, 7.15042384e-03, 7.79300062e+00,
        7.81055000e-01, 1.22449088e+00],
       [1.94215894e+01, 3.13888651e+01, 7.15042384e-03, 7.79300062e+00,
        7.81055000e-01, 1.22449088e+00]])>

In [68]:
ess_model_exper_fixed_rt_community_params_cal_1 = tfp.mcmc.effective_sample_size(states_model_exper_fixed_rt_community_params_cal_1  )
print(ess_model_exper_fixed_rt_community_params_cal_1)
ess_model_exper_fixed_rt_community_params_cal_2 = tfp.mcmc.effective_sample_size(states_model_exper_fixed_rt_community_params_cal_2  )
print(ess_model_exper_fixed_rt_community_params_cal_2)
ess_model_exper_fixed_rt_community_params_cal_3 = tfp.mcmc.effective_sample_size(states_model_exper_fixed_rt_community_params_cal_3  )
print(ess_model_exper_fixed_rt_community_params_cal_3)
ess_model_exper_fixed_rt_community_params_cal_4 = tfp.mcmc.effective_sample_size(states_model_exper_fixed_rt_community_params_cal_4  )
print(ess_model_exper_fixed_rt_community_params_cal_4)

tf.Tensor(
[3420.32860121 3550.61376494 3182.59029981 3320.44479116 3491.96250117
 3623.68014361], shape=(6,), dtype=float64)
tf.Tensor(
[3785.41496537 3720.7704649  3596.5281548  2928.85841137 4240.88332564
 3542.01175854], shape=(6,), dtype=float64)
tf.Tensor(
[3822.10194387 3754.64174815 3870.5925267  3052.38126513 3826.65120109
 3119.18160647], shape=(6,), dtype=float64)
tf.Tensor(
[3796.53008838 3581.59219929 3475.36997732 3237.27187796 3984.34185068
 3537.87496593], shape=(6,), dtype=float64)


In [69]:
print('Min 1:', min(ess_model_exper_fixed_rt_community_params_cal_1))
print('Min 2:', min(ess_model_exper_fixed_rt_community_params_cal_2))
print('Min 3:', min(ess_model_exper_fixed_rt_community_params_cal_3))
print('Min 4:', min(ess_model_exper_fixed_rt_community_params_cal_4))

print('Min all', min((min(ess_model_exper_fixed_rt_community_params_cal_1), min(ess_model_exper_fixed_rt_community_params_cal_2), min(ess_model_exper_fixed_rt_community_params_cal_3), min(ess_model_exper_fixed_rt_community_params_cal_4))))

Min 1: tf.Tensor(3182.590299807659, shape=(), dtype=float64)
Min 2: tf.Tensor(2928.8584113653305, shape=(), dtype=float64)
Min 3: tf.Tensor(3052.3812651289813, shape=(), dtype=float64)
Min 4: tf.Tensor(3237.2718779576744, shape=(), dtype=float64)
Min all tf.Tensor(2928.8584113653305, shape=(), dtype=float64)


#### Model Exper w/Adapted LL

In [70]:
states_model_exper_adapted_LL_1 = tf.convert_to_tensor(model_exper_adapted_LL_chain_1_burn_in)
states_model_exper_adapted_LL_2 = tf.convert_to_tensor(model_exper_adapted_LL_chain_3_burn_in)
states_model_exper_adapted_LL_3 = tf.convert_to_tensor(model_exper_adapted_LL_chain_3_burn_in)
states_model_exper_adapted_LL_4 = tf.convert_to_tensor(model_exper_adapted_LL_chain_4_burn_in)

In [71]:
states_model_exper_adapted_LL_1

<tf.Tensor: shape=(30000, 9), dtype=float64, numpy=
array([[ 8.        , 12.78206697, 27.27161575, ...,  0.88593305,
         1.21930238, 23.        ],
       [ 8.        , 12.78206697, 27.27161575, ...,  0.88593305,
         1.21930238, 23.        ],
       [ 8.        , 12.78206697, 27.27161575, ...,  0.88593305,
         1.21930238, 23.        ],
       ...,
       [ 8.        , 15.92769238, 31.42696228, ...,  0.73182971,
         1.22002934, 14.        ],
       [ 8.        , 15.92769238, 31.42696228, ...,  0.73182971,
         1.22002934, 14.        ],
       [ 8.        , 15.92769238, 31.42696228, ...,  0.73182971,
         1.22002934, 14.        ]])>

In [72]:
ess_model_exper_adapted_LL_1 = tfp.mcmc.effective_sample_size(states_model_exper_adapted_LL_1  )
print(ess_model_exper_adapted_LL_1)
ess_model_exper_adapted_LL_2 = tfp.mcmc.effective_sample_size(states_model_exper_adapted_LL_2  )
print(ess_model_exper_adapted_LL_2)
ess_model_exper_adapted_LL_3 = tfp.mcmc.effective_sample_size(states_model_exper_adapted_LL_3  )
print(ess_model_exper_adapted_LL_3)
ess_model_exper_adapted_LL_4 = tfp.mcmc.effective_sample_size(states_model_exper_adapted_LL_4  )
print(ess_model_exper_adapted_LL_4)

tf.Tensor(
[431.69392346 389.23568211 331.79153737 347.30222667 439.22387244
 363.48639377 380.09196869 349.34110878 302.04270744], shape=(9,), dtype=float64)
tf.Tensor(
[364.16464707 308.67942301 309.97740887 247.96169686 409.47845028
 425.1933003  400.40183325 375.08878452 328.82622937], shape=(9,), dtype=float64)
tf.Tensor(
[364.16464707 308.67942301 309.97740887 247.96169686 409.47845028
 425.1933003  400.40183325 375.08878452 328.82622937], shape=(9,), dtype=float64)
tf.Tensor(
[193.94674306 159.02620797 133.03904877 166.71315618 192.42595435
 232.60288401 185.61122404 147.59591769 195.23203196], shape=(9,), dtype=float64)


In [73]:
print('Min 1:', min(ess_model_exper_adapted_LL_1))
print('Min 2:', min(ess_model_exper_adapted_LL_2))
print('Min 3:', min(ess_model_exper_adapted_LL_3))
print('Min 4:', min(ess_model_exper_adapted_LL_4))

print('Min All', min((min(ess_model_exper_adapted_LL_1), min(ess_model_exper_adapted_LL_2), min(ess_model_exper_adapted_LL_3), min(ess_model_exper_adapted_LL_4))))

Min 1: tf.Tensor(302.0427074366912, shape=(), dtype=float64)
Min 2: tf.Tensor(247.961696855389, shape=(), dtype=float64)
Min 3: tf.Tensor(247.961696855389, shape=(), dtype=float64)
Min 4: tf.Tensor(133.0390487724828, shape=(), dtype=float64)
Min All tf.Tensor(133.0390487724828, shape=(), dtype=float64)


#### RIT Data w/Adapted LL

In [74]:
states_rit_data_adapted_LL_1 = tf.convert_to_tensor(rit_data_adapted_LL_chain_1_burn_in)
# states_rit_data_adapted_LL_2 = tf.convert_to_tensor(rit_data_adapted_LL_chain_2_burn_in)
states_rit_data_adapted_LL_3 = tf.convert_to_tensor(rit_data_adapted_LL_chain_3_burn_in)
states_rit_data_adapted_LL_4 = tf.convert_to_tensor(rit_data_adapted_LL_chain_4_burn_in)

In [75]:
states_rit_data_adapted_LL_1

<tf.Tensor: shape=(140000, 9), dtype=float64, numpy=
array([[ 6.        , 12.05787581,  6.42939387, ...,  0.76195621,
         1.22723661,  7.        ],
       [ 6.        , 12.05787581,  6.42939387, ...,  0.76195621,
         1.22723661,  7.        ],
       [ 6.        , 12.05787581,  6.42939387, ...,  0.76195621,
         1.22723661,  7.        ],
       ...,
       [ 8.        , 11.49791581,  6.25027808, ...,  0.86248248,
         1.23497916, 11.        ],
       [ 8.        , 11.49791581,  6.25027808, ...,  0.86248248,
         1.23497916, 11.        ],
       [ 8.        , 11.49791581,  6.25027808, ...,  0.86248248,
         1.23497916, 11.        ]])>

In [76]:
ess_rit_data_adapted_LL_1 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_1  )
print(ess_rit_data_adapted_LL_1)
ess_rit_data_adapted_LL_3 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_3  )
print(ess_rit_data_adapted_LL_3)
ess_rit_data_adapted_LL_4 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_4  )
print(ess_rit_data_adapted_LL_4)

tf.Tensor(
[1744.60698726 1112.0634369  2152.14829536 1348.1308908  1599.01619877
 1388.88354243 1023.9545827  1569.69837269 2030.18751373], shape=(9,), dtype=float64)
tf.Tensor(
[2235.30580998 1967.41867091 2557.97982823 2460.52897873 2559.45206431
 2433.44607346 2162.91318255 2669.63414452 2134.91729862], shape=(9,), dtype=float64)
tf.Tensor(
[2305.32624524 1977.11801947 2882.34339366 1933.22845993 2523.18006627
 2219.26614796 2698.01601004 2801.18382628 2198.04933947], shape=(9,), dtype=float64)


In [77]:
print('Min 1:', min(ess_rit_data_adapted_LL_1))
print('Min 3:', min(ess_rit_data_adapted_LL_3))
print('Min 4:', min(ess_rit_data_adapted_LL_4))

print('Min all:', min((min(ess_rit_data_adapted_LL_1), min(ess_rit_data_adapted_LL_3), min(ess_rit_data_adapted_LL_4))))

Min 1: tf.Tensor(1023.9545826970644, shape=(), dtype=float64)
Min 3: tf.Tensor(1967.4186709108271, shape=(), dtype=float64)
Min 4: tf.Tensor(1933.2284599333432, shape=(), dtype=float64)
Min all: tf.Tensor(1023.9545826970644, shape=(), dtype=float64)


#### RIT Data w/Adapted LL and Initially Infected

In [78]:
states_rit_data_adapted_LL_init_infect_1 = tf.convert_to_tensor(rit_data_adapted_LL_init_infect_chain_1_burn_in)
states_rit_data_adapted_LL_init_infect_2 = tf.convert_to_tensor(rit_data_adapted_LL_init_infect_chain_2_burn_in)
states_rit_data_adapted_LL_init_infect_3 = tf.convert_to_tensor(rit_data_adapted_LL_init_infect_chain_3_burn_in)
states_rit_data_adapted_LL_init_infect_4 = tf.convert_to_tensor(rit_data_adapted_LL_init_infect_chain_4_burn_in)

In [79]:
states_rit_data_adapted_LL_init_infect_1

<tf.Tensor: shape=(440000, 10), dtype=float64, numpy=
array([[ 8.        , 10.72856915,  7.61959615, ...,  1.22826045,
        13.        ,  4.        ],
       [ 8.        , 10.72856915,  7.61959615, ...,  1.22826045,
        13.        ,  4.        ],
       [ 8.        , 10.72856915,  7.61959615, ...,  1.22826045,
        13.        ,  4.        ],
       ...,
       [ 7.        , 12.59645808, 10.59498471, ...,  1.23605017,
         7.        ,  6.        ],
       [ 7.        , 12.59645808, 10.59498471, ...,  1.23605017,
         7.        ,  6.        ],
       [ 7.        , 12.59645808, 10.59498471, ...,  1.23605017,
         7.        ,  6.        ]])>

In [80]:
ess_rit_data_adapted_LL_init_infect_1 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_init_infect_1  )
print(ess_rit_data_adapted_LL_init_infect_1)
ess_rit_data_adapted_LL_init_infect_2 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_init_infect_2  )
print(ess_rit_data_adapted_LL_init_infect_2)
ess_rit_data_adapted_LL_init_infect_3 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_init_infect_3  )
print(ess_rit_data_adapted_LL_init_infect_3)
ess_rit_data_adapted_LL_init_infect_4 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_init_infect_4  )
print(ess_rit_data_adapted_LL_init_infect_4)

tf.Tensor(
[5696.94489237 4736.11143982 5348.93732655 5175.50945936 4835.16161265
 5827.01714083 6639.07975398 5271.89588094 4223.58346545 4994.54934243], shape=(10,), dtype=float64)
tf.Tensor(
[5988.11544218 4817.81836249 4569.11482424 4901.22223589 6008.77170288
 6143.48534055 6702.9187415  5963.10763021 4741.00398831 5913.5488761 ], shape=(10,), dtype=float64)
tf.Tensor(
[6009.55465272 5528.73515299 3873.76382275 5136.24869164 5982.83637771
 6314.24591062 6168.32749806 5455.73726545 4134.87147301 5954.99209199], shape=(10,), dtype=float64)
tf.Tensor(
[5172.64873774 5581.54560449 4496.09662311 5383.31294431 5430.83332076
 5184.88930899 5936.86345528 4794.71920762 3962.00426655 5361.17142422], shape=(10,), dtype=float64)


In [81]:
print('Min 1:', min(ess_rit_data_adapted_LL_init_infect_1))
print('Min 2:', min(ess_rit_data_adapted_LL_init_infect_2))
print('Min 3:', min(ess_rit_data_adapted_LL_init_infect_3))
print('Min 4:', min(ess_rit_data_adapted_LL_init_infect_4))

print('Min all:', min((min(ess_rit_data_adapted_LL_init_infect_1), min(ess_rit_data_adapted_LL_init_infect_2), min(ess_rit_data_adapted_LL_init_infect_3), min(ess_rit_data_adapted_LL_init_infect_4))))

Min 1: tf.Tensor(4223.583465453808, shape=(), dtype=float64)
Min 2: tf.Tensor(4569.114824238087, shape=(), dtype=float64)
Min 3: tf.Tensor(3873.76382274878, shape=(), dtype=float64)
Min 4: tf.Tensor(3962.00426654719, shape=(), dtype=float64)
Min all: tf.Tensor(3873.76382274878, shape=(), dtype=float64)


#### RIT Data w/Adapted LL weighted LL Stochastic tests initial iso

In [82]:
states_rit_data_adapted_LL_stochastic_weight_1 = tf.convert_to_tensor(rit_data_adapted_LL_init_infect_stochastic_weight_chain_1_burn_in)
states_rit_data_adapted_LL_stochastic_weight_2 = tf.convert_to_tensor(rit_data_adapted_LL_init_infect_stochastic_weight_chain_2_burn_in)
states_rit_data_adapted_LL_stochastic_weight_3 = tf.convert_to_tensor(rit_data_adapted_LL_init_infect_stochastic_weight_chain_3_burn_in)
states_rit_data_adapted_LL_stochastic_weight_4 = tf.convert_to_tensor(rit_data_adapted_LL_init_infect_stochastic_weight_chain_4_burn_in)

In [83]:
states_rit_data_adapted_LL_stochastic_weight_4

<tf.Tensor: shape=(380000, 11), dtype=float64, numpy=
array([[ 7.        , 16.16128881, 18.74643165, ..., 13.        ,
         4.        ,  1.07179497],
       [ 7.        , 16.16128881, 18.74643165, ..., 13.        ,
         4.        ,  1.07179497],
       [ 7.        , 16.16128881, 18.74643165, ..., 13.        ,
         4.        ,  1.07179497],
       ...,
       [ 8.        , 14.11698249, 14.70025621, ..., 16.        ,
         7.        ,  1.66465942],
       [ 8.        , 14.11698249, 14.70025621, ..., 16.        ,
         7.        ,  1.66465942],
       [ 8.        , 14.11698249, 14.70025621, ..., 16.        ,
         7.        ,  1.66465942]])>

In [84]:
ess_rit_data_adapted_LL_stochastic_weight_1 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_stochastic_weight_1 )
print(ess_rit_data_adapted_LL_stochastic_weight_1)
ess_rit_data_adapted_LL_stochastic_weight_2 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_stochastic_weight_2  )
print(ess_rit_data_adapted_LL_stochastic_weight_2)
ess_rit_data_adapted_LL_stochastic_weight_3 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_stochastic_weight_3  )
print(ess_rit_data_adapted_LL_stochastic_weight_3)
ess_rit_data_adapted_LL_stochastic_weight_4 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_stochastic_weight_4  )
print(ess_rit_data_adapted_LL_stochastic_weight_4)

tf.Tensor(
[4904.64876999 4034.42676216 5544.55032144 3331.2304897  3819.10665194
 4121.45402105 4189.52459303 4132.57814758 2964.57667199 4068.05665002
 4143.7893942 ], shape=(11,), dtype=float64)
tf.Tensor(
[4680.84052592 4642.64558246 4435.45526853 2841.36238118 4625.54692106
 4076.05792192 4331.26802249 4828.38264423 3299.72152682 4700.61566613
 4855.43142636], shape=(11,), dtype=float64)
tf.Tensor(
[4977.84365326 4735.74111512 5216.16721265 3710.47152162 4477.03133626
 4485.25836054 4960.39592057 5002.2835565  3475.36166182 4575.89258094
 4645.98738579], shape=(11,), dtype=float64)
tf.Tensor(
[5243.66148132 4685.65133426 3945.31627166 3969.10254045 4521.59865687
 4723.94905452 4871.58607923 4284.22306596 4039.19506721 5010.21024973
 5092.38270262], shape=(11,), dtype=float64)


In [85]:
print('Min 1:', min(ess_rit_data_adapted_LL_stochastic_weight_1))
print('Min 2:', min(ess_rit_data_adapted_LL_stochastic_weight_2))
print('Min 3:', min(ess_rit_data_adapted_LL_stochastic_weight_3))
print('Min 4:', min(ess_rit_data_adapted_LL_stochastic_weight_4))

print('Min all:', min((min(ess_rit_data_adapted_LL_stochastic_weight_1), min(ess_rit_data_adapted_LL_stochastic_weight_2), min(ess_rit_data_adapted_LL_stochastic_weight_3), min(ess_rit_data_adapted_LL_stochastic_weight_4))))

Min 1: tf.Tensor(2964.5766719927506, shape=(), dtype=float64)
Min 2: tf.Tensor(2841.362381179028, shape=(), dtype=float64)
Min 3: tf.Tensor(3475.361661815453, shape=(), dtype=float64)
Min 4: tf.Tensor(3945.3162716553775, shape=(), dtype=float64)
Min all: tf.Tensor(2841.362381179028, shape=(), dtype=float64)


#### RIT Data w/Adapted LL and Initially Infected

In [78]:
states_rit_data_adapted_LL_init_infect_1 = tf.convert_to_tensor(rit_data_adapted_LL_init_infect_chain_1_burn_in)
states_rit_data_adapted_LL_init_infect_2 = tf.convert_to_tensor(rit_data_adapted_LL_init_infect_chain_2_burn_in)
states_rit_data_adapted_LL_init_infect_3 = tf.convert_to_tensor(rit_data_adapted_LL_init_infect_chain_3_burn_in)
states_rit_data_adapted_LL_init_infect_4 = tf.convert_to_tensor(rit_data_adapted_LL_init_infect_chain_4_burn_in)

In [79]:
states_rit_data_adapted_LL_init_infect_1

<tf.Tensor: shape=(440000, 10), dtype=float64, numpy=
array([[ 8.        , 10.72856915,  7.61959615, ...,  1.22826045,
        13.        ,  4.        ],
       [ 8.        , 10.72856915,  7.61959615, ...,  1.22826045,
        13.        ,  4.        ],
       [ 8.        , 10.72856915,  7.61959615, ...,  1.22826045,
        13.        ,  4.        ],
       ...,
       [ 7.        , 12.59645808, 10.59498471, ...,  1.23605017,
         7.        ,  6.        ],
       [ 7.        , 12.59645808, 10.59498471, ...,  1.23605017,
         7.        ,  6.        ],
       [ 7.        , 12.59645808, 10.59498471, ...,  1.23605017,
         7.        ,  6.        ]])>

In [80]:
ess_rit_data_adapted_LL_init_infect_1 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_init_infect_1  )
print(ess_rit_data_adapted_LL_init_infect_1)
ess_rit_data_adapted_LL_init_infect_2 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_init_infect_2  )
print(ess_rit_data_adapted_LL_init_infect_2)
ess_rit_data_adapted_LL_init_infect_3 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_init_infect_3  )
print(ess_rit_data_adapted_LL_init_infect_3)
ess_rit_data_adapted_LL_init_infect_4 = tfp.mcmc.effective_sample_size(states_rit_data_adapted_LL_init_infect_4  )
print(ess_rit_data_adapted_LL_init_infect_4)

tf.Tensor(
[5696.94489237 4736.11143982 5348.93732655 5175.50945936 4835.16161265
 5827.01714083 6639.07975398 5271.89588094 4223.58346545 4994.54934243], shape=(10,), dtype=float64)
tf.Tensor(
[5988.11544218 4817.81836249 4569.11482424 4901.22223589 6008.77170288
 6143.48534055 6702.9187415  5963.10763021 4741.00398831 5913.5488761 ], shape=(10,), dtype=float64)
tf.Tensor(
[6009.55465272 5528.73515299 3873.76382275 5136.24869164 5982.83637771
 6314.24591062 6168.32749806 5455.73726545 4134.87147301 5954.99209199], shape=(10,), dtype=float64)
tf.Tensor(
[5172.64873774 5581.54560449 4496.09662311 5383.31294431 5430.83332076
 5184.88930899 5936.86345528 4794.71920762 3962.00426655 5361.17142422], shape=(10,), dtype=float64)


In [81]:
print('Min 1:', min(ess_rit_data_adapted_LL_init_infect_1))
print('Min 2:', min(ess_rit_data_adapted_LL_init_infect_2))
print('Min 3:', min(ess_rit_data_adapted_LL_init_infect_3))
print('Min 4:', min(ess_rit_data_adapted_LL_init_infect_4))

print('Min all:', min((min(ess_rit_data_adapted_LL_init_infect_1), min(ess_rit_data_adapted_LL_init_infect_2), min(ess_rit_data_adapted_LL_init_infect_3), min(ess_rit_data_adapted_LL_init_infect_4))))

Min 1: tf.Tensor(4223.583465453808, shape=(), dtype=float64)
Min 2: tf.Tensor(4569.114824238087, shape=(), dtype=float64)
Min 3: tf.Tensor(3873.76382274878, shape=(), dtype=float64)
Min 4: tf.Tensor(3962.00426654719, shape=(), dtype=float64)
Min all: tf.Tensor(3873.76382274878, shape=(), dtype=float64)


### Define Needed Functions for ESS

In [ ]:
from statsmodels.tsa.stattools import acf


In [ ]:
def acf_all_params_num_lags(burned_in_df, num_lags):
    acf_all_params_output = []
    
    for i in range(len(burned_in_df[0,:])):
        print(i, burned_in_df[:,i],num_lags[i] )
        acf_all_params_output.append(acf(burned_in_df[:,i], nlags = num_lags[i]))
    
    return(acf_all_params_output)

In [ ]:
def acf_all_params_num_lags_nlag_det(burned_in_df, num_lags):
    acf_all_params_output = []
    
    for i in range(len(burned_in_df[0,:])):
        print(burned_in_df[:,i])
        acf_all_params_output.append(acf(burned_in_df[:,i], nlags = num_lags))
    
    return(acf_all_params_output)

In [ ]:
def ess_all_params(acf_outputs, num_samples):
    ess_output_all_params = []
    
    for i in range(len(acf_outputs)):
        ess = num_samples/(1+(2*sum(acf_outputs[i])))
        ess_output_all_params.append(ess)
        
    return(ess_output_all_params)

In [ ]:
def ess_to_dataframe(ess_outputs_for_params, param_names, index_name):
    
    ess_outputs_for_params = np.resize(ess_outputs_for_params, (1,len(param_names)))

    ess_df = pd.DataFrame(ess_outputs_for_params, index = index_name, columns = param_names)

    return(ess_df)

### Testing Number of Lags for ACF function
#### OG Calibration

In [ ]:
numlags = 500


acf_lag_output_og_cal = acf_all_params_num_lags_nlag_det(og_calibration_burn_in, num_lags= numlags)

In [ ]:
def det_num_lags(acf_lag_output_input, param_index):
        
    for i in range(1, len(acf_lag_output_input[0])):
        
        if ((acf_lag_output_input[param_index][i - 1] > 0 and  acf_lag_output_input[param_index][i] < 0) == True).any():
            print(i-1, i, acf_lag_output_input[param_index][i - 1], acf_lag_output_input[param_index][i])

##### Frequency of Exgenous Shock Parameter

In [ ]:
det_num_lags(acf_lag_output_og_cal, param_index = 0)

In [ ]:
freq_es_nlag = 198
acf_lag_output_og_cal[0][198]


In [ ]:
param_numlags_og_cal = []
param_numlags_og_cal.append(freq_es_nlag)


##### Time to Recovery Parameter

In [ ]:
det_num_lags(acf_lag_output_og_cal, param_index = 1)

In [ ]:
time_recov_nlags = 450
acf_lag_output_og_cal[1][450]


In [ ]:
param_numlags_og_cal.append(time_recov_nlags)
param_numlags_og_cal

##### Percent Advancing to Symptoms Parameter

In [ ]:
det_num_lags(acf_lag_output_og_cal, param_index = 2)

In [ ]:
numlags = 500


acf_lag_output_og_cal_1 = acf_all_params_num_lags_nlag_det(og_calibration_burn_in, num_lags= numlags)

In [ ]:
det_num_lags(acf_lag_output_og_cal_1, param_index = 2)

In [ ]:
percent_advance_nlags = 245
acf_lag_output_og_cal[2][245]

In [ ]:
param_numlags_og_cal.append(percent_advance_nlags)
param_numlags_og_cal

##### $R_t$ Parameter

In [ ]:
det_num_lags(acf_lag_output_og_cal_1, param_index = 3)

In [ ]:
Rt_nlags = 251
acf_lag_output_og_cal[3][251]

In [ ]:
param_numlags_og_cal.append(Rt_nlags)
param_numlags_og_cal

##### Symptom Case Fatality Ratio Parameter

In [ ]:
det_num_lags(acf_lag_output_og_cal_1, param_index = 4)

In [ ]:
symptom_case_fatal_nlags = 175
acf_lag_output_og_cal[4][175]

In [ ]:
param_numlags_og_cal.append(symptom_case_fatal_nlags)
param_numlags_og_cal

##### Days to Incubation Parameter

In [ ]:
det_num_lags(acf_lag_output_og_cal_1, param_index = 5)

In [ ]:
acf_lag_output_og_cal[5][156]

In [ ]:
numlags = 200


acf_lag_output_og_cal_2 = acf_all_params_num_lags_nlag_det(og_calibration_burn_in, num_lags= numlags)

In [ ]:
det_num_lags(acf_lag_output_og_cal_2, param_index = 5)

In [ ]:
days_nlags = 156
acf_lag_output_og_cal[5][156]

In [ ]:
param_numlags_og_cal.append(days_nlags)
param_numlags_og_cal

##### Test Sensitivity Parameter

In [ ]:
det_num_lags(acf_lag_output_og_cal, param_index = 6)

In [ ]:
acf_lag_output_og_cal[6][125]

In [ ]:
det_num_lags(acf_lag_output_og_cal_1, param_index = 6)

In [ ]:
Se_nlags = 125
acf_lag_output_og_cal[6][125]

In [ ]:
param_numlags_og_cal.append(Se_nlags)
param_numlags_og_cal

##### Test Specificity Parameter

In [ ]:
det_num_lags(acf_lag_output_og_cal, param_index = 7)

In [ ]:
Sp_nlags = 207
acf_lag_output_og_cal[7][207]

In [ ]:
param_numlags_og_cal.append(Sp_nlags)
param_numlags_og_cal

##### New Infections Per Shock Parameter

In [ ]:
det_num_lags(acf_lag_output_og_cal, param_index = 8)

In [ ]:
new_infect_nlags = 226
acf_lag_output_og_cal[8][226]

In [ ]:
param_numlags_og_cal.append(new_infect_nlags)
param_numlags_og_cal

#### Initial Infected Adjusted Testing Term 

In [ ]:
numlags = 500


acf_lag_output_init_adjust_tt_cal = acf_all_params_num_lags_nlag_det(initi_infect_adjust_test_terms_burn_in, num_lags= numlags)


##### Frequency of Exgenous Shock Parameter

In [ ]:
det_num_lags(acf_lag_output_init_adjust_tt_cal, param_index = 0)

In [ ]:
freq_es_nlag = 330
acf_lag_output_init_adjust_tt_cal[0][330]

In [ ]:
param_numlags_init_adjust_tt_cal = []
param_numlags_init_adjust_tt_cal.append(freq_es_nlag)

##### Time to Recovery Parameter

In [ ]:
det_num_lags(acf_lag_output_init_adjust_tt_cal, param_index = 1)

In [ ]:
time_recov_nlag = 199
acf_lag_output_init_adjust_tt_cal[1][199]

In [ ]:
param_numlags_init_adjust_tt_cal.append(time_recov_nlag)
param_numlags_init_adjust_tt_cal

##### Percent Advancing to Symptoms Parameter

In [ ]:
det_num_lags(acf_lag_output_init_adjust_tt_cal, param_index = 2)

In [ ]:
# numlags = 20000


# acf_lag_output_init_adjust_tt_cal_1 = acf_all_params_num_lags_nlag_det(initi_infect_adjust_test_terms_burn_in, num_lags= numlags)



In [ ]:
# det_num_lags(acf_lag_output_init_adjust_tt_cal_1, param_index = 2)

In [ ]:
percent_advanc_nlag = 234
acf_lag_output_init_adjust_tt_cal[2][234]

In [ ]:
param_numlags_init_adjust_tt_cal.append(percent_advanc_nlag)
param_numlags_init_adjust_tt_cal

##### $R_t$ Parameter

In [ ]:
det_num_lags(acf_lag_output_init_adjust_tt_cal, param_index = 3)

In [ ]:
Rt_nlag = 262
acf_lag_output_init_adjust_tt_cal[3][262]

In [ ]:
param_numlags_init_adjust_tt_cal.append(Rt_nlag)
param_numlags_init_adjust_tt_cal

##### Symptom Case Fatality Ratio Parameter

In [ ]:
det_num_lags(acf_lag_output_init_adjust_tt_cal, param_index = 4)

In [ ]:
symptom_case_fatal_nlag = 130
acf_lag_output_init_adjust_tt_cal[4][130]

In [ ]:
param_numlags_init_adjust_tt_cal.append(symptom_case_fatal_nlag)
param_numlags_init_adjust_tt_cal

##### Days to Incubation Parameter

In [ ]:
det_num_lags(acf_lag_output_init_adjust_tt_cal, param_index = 5)

In [ ]:
days_nlag = 222
acf_lag_output_init_adjust_tt_cal[5][222]

In [ ]:
param_numlags_init_adjust_tt_cal.append(days_nlag)
param_numlags_init_adjust_tt_cal

##### Test Sensitivity Parameter

In [ ]:
det_num_lags(acf_lag_output_init_adjust_tt_cal, param_index = 6)

In [ ]:
Se_nlag = 133
acf_lag_output_init_adjust_tt_cal[6][133]

In [ ]:
param_numlags_init_adjust_tt_cal.append(Se_nlag)
param_numlags_init_adjust_tt_cal

##### Test Specificity Parameter

In [ ]:
det_num_lags(acf_lag_output_init_adjust_tt_cal, param_index = 7)

In [ ]:
Sp_nlag = 96
acf_lag_output_init_adjust_tt_cal[7][96]

In [ ]:
param_numlags_init_adjust_tt_cal.append(Sp_nlag)
param_numlags_init_adjust_tt_cal

##### New Infections Per Shock Parameter

In [ ]:
det_num_lags(acf_lag_output_init_adjust_tt_cal, param_index = 8)

In [ ]:
new_infect_nlag = 202
acf_lag_output_init_adjust_tt_cal[8][202]

In [ ]:
param_numlags_init_adjust_tt_cal.append(new_infect_nlag)
param_numlags_init_adjust_tt_cal

##### Initially Infected Parameter

In [ ]:
det_num_lags(acf_lag_output_init_adjust_tt_cal, param_index = 9)

In [ ]:
init_infect_nlag = 450
acf_lag_output_init_adjust_tt_cal[9][450]

In [ ]:
param_numlags_init_adjust_tt_cal.append(init_infect_nlag)
param_numlags_init_adjust_tt_cal

#### Sp Reparameter Calibration

In [ ]:
numlags = 10000


acf_lag_output_reparam_cal = acf_all_params_num_lags_nlag_det(sp_reparam_burn_in, num_lags= numlags)

##### Frequency of Exgenous Shock Parameter

In [ ]:
det_num_lags(acf_lag_output_reparam_cal, param_index = 0)

In [ ]:
freq_es_nlag = 9589
acf_lag_output_reparam_cal[0][9589]

In [ ]:
param_numlags_reparam_cal = []
param_numlags_reparam_cal.append(freq_es_nlag)

##### Time to Recovery Parameter

In [ ]:
det_num_lags(acf_lag_output_reparam_cal, param_index = 1)

In [ ]:
numlags = 20000


acf_lag_output_reparam_cal_1 = acf_all_params_num_lags_nlag_det(sp_reparam_burn_in, num_lags= numlags)



In [ ]:
det_num_lags(acf_lag_output_reparam_cal_1, param_index = 1)

In [ ]:
time_recov_nlag = 10211
acf_lag_output_reparam_cal_1[1][10211]

In [ ]:
param_numlags_reparam_cal.append(time_recov_nlag)
param_numlags_reparam_cal

##### Percent Advancing to Symptoms Parameter

In [ ]:
det_num_lags(acf_lag_output_reparam_cal, param_index = 2)

In [ ]:
det_num_lags(acf_lag_output_reparam_cal_1, param_index = 2)

In [ ]:
percent_advance_nlag = 10221
acf_lag_output_reparam_cal_1[2][10221]

In [ ]:
param_numlags_reparam_cal.append(percent_advance_nlag)
param_numlags_reparam_cal

##### $R_t$ Parameter

In [ ]:
det_num_lags(acf_lag_output_reparam_cal, param_index = 3)

In [ ]:
Rt_nlag = 4047
acf_lag_output_reparam_cal_1[3][4047]

In [ ]:
param_numlags_reparam_cal.append(Rt_nlag)
param_numlags_reparam_cal

##### Symptom Case Fatality Ratio Parameter

In [ ]:
det_num_lags(acf_lag_output_reparam_cal, param_index = 4)

In [ ]:
symptom_case_fatal_nlag = 4882
acf_lag_output_reparam_cal_1[4][4882]

In [ ]:
param_numlags_reparam_cal.append(symptom_case_fatal_nlag)
param_numlags_reparam_cal

##### Days to Incubation Parameter

In [ ]:
det_num_lags(acf_lag_output_reparam_cal, param_index = 5)

In [ ]:
days_nlag = 3644
acf_lag_output_reparam_cal_1[5][3644]

In [ ]:
param_numlags_reparam_cal.append(days_nlag)
param_numlags_reparam_cal

##### Test Sensitivity Parameter

In [ ]:
det_num_lags(acf_lag_output_reparam_cal, param_index = 6)

In [ ]:
Se_nlag = 5372
acf_lag_output_reparam_cal_1[6][5372]

In [ ]:
param_numlags_reparam_cal.append(Se_nlag)
param_numlags_reparam_cal

##### Test Specificity Parameter

In [ ]:
det_num_lags(acf_lag_output_reparam_cal, param_index = 7)

In [ ]:
Sp_nlag = 3439
acf_lag_output_reparam_cal_1[7][3439]

In [ ]:
param_numlags_reparam_cal.append(Sp_nlag)
param_numlags_reparam_cal

##### New Infections Per Shock Parameter

In [ ]:
det_num_lags(acf_lag_output_reparam_cal, param_index = 8)

In [ ]:
det_num_lags(acf_lag_output_reparam_cal_1, param_index = 8)

In [ ]:
new_infect_nlag = 11310
acf_lag_output_reparam_cal_1[8][11310]

In [ ]:
param_numlags_reparam_cal.append(new_infect_nlag)
param_numlags_reparam_cal

##### Initially Infected Parameter

In [ ]:
det_num_lags(acf_lag_output_reparam_cal, param_index = 9)

In [ ]:
det_num_lags(acf_lag_output_reparam_cal_1, param_index = 9)

In [ ]:
init_infect_nlag = 11310
acf_lag_output_reparam_cal_1[9][11310]

In [ ]:
param_numlags_reparam_cal.append(init_infect_nlag)
param_numlags_reparam_cal

##### Delay Parameter

In [ ]:
det_num_lags(acf_lag_output_reparam_cal, param_index = 10)

In [ ]:
delay_nlag = 1793
acf_lag_output_reparam_cal_1[10][1793]

In [ ]:
param_numlags_reparam_cal.append(delay_nlag)
param_numlags_reparam_cal

##### Non-compliance Parameter

In [ ]:
det_num_lags(acf_lag_output_reparam_cal, param_index = 11)

In [ ]:
non_comp_nlag = 8551
acf_lag_output_reparam_cal_1[11][8551]

In [ ]:
param_numlags_reparam_cal.append(non_comp_nlag)
param_numlags_reparam_cal

### Calculate ESS
#### OG Calibration

In [ ]:
param_numlags_og_cal

In [ ]:
param_numlags_og_cal[8]

In [ ]:
acf_og_burn_in = acf_all_params_num_lags(og_calibration_burn_in, param_numlags_og_cal )

In [ ]:
ess_og_burn_in = ess_all_params(acf_og_burn_in, num_samples=len(og_calibration_burn_in[:,0]))

In [ ]:
ess_og_burn_in

In [ ]:
og_names

In [ ]:
og_ess_df = ess_to_dataframe(ess_og_burn_in, og_names, index_name = ['og calibration'])
og_ess_df

In [ ]:
ess_og_cal = tfp.mcmc.effective_sample_size(states_og_cal)
print(ess_og_cal)

#### Initial Infected and Adjusted Testing Term Calibration

In [ ]:
acf_init_infect_adjust_tt_burn_in = acf_all_params_num_lags(initi_infect_adjust_test_terms_burn_in, param_numlags_init_adjust_tt_cal)


In [ ]:
ess_init_infect_adjust_tt_burn_in = ess_all_params(acf_init_infect_adjust_tt_burn_in, num_samples=len(initi_infect_adjust_test_terms_burn_in[:,0]))


In [ ]:
init_adjust_tt_ess_df = ess_to_dataframe(ess_init_infect_adjust_tt_burn_in, init_infect_adjust_tt_names, index_name = ['initial infect adjust tt calibration'])
init_adjust_tt_ess_df

In [ ]:
ess_init_cal = tfp.mcmc.effective_sample_size(states_init_cal)
print(ess_init_cal)

#### Sp Reparam Calibration

In [ ]:
acf_sp_reparam_burn_in = acf_all_params_num_lags(sp_reparam_burn_in, param_numlags_reparam_cal)


In [ ]:
ess_sp_reparam_burn_in = ess_all_params(acf_sp_reparam_burn_in, num_samples=len(sp_reparam_burn_in[:,0]))


In [ ]:
sp_reparam_ess_df = ess_to_dataframe(ess_sp_reparam_burn_in, sp_reparam_names, index_name = ['sp reparam calibration'])
sp_reparam_ess_df